# Deep Q-learner starter code

**Work in progress!** Please forgive lack of clarity, bugs, and typos.

This notebook aims to demonstrate creating a deep Q-learner using Keras and train it on the GFootball enviroment. It will focus on the coded need to train a deep q-learner agent, rather than theory. Hopefully it will provide a starting point developing models and testing them out on the GFootball enviroment, and show how to train and evaluate them using the Gym API. I have no idea at this point how well a Q-learner is likely to perform in this competition, though! It'll certainly take a lot of training and tweaking, and and a large replay buffer to perform well. See also [Convolutional Deep-Q learner](https://www.kaggle.com/garethjns/convolutional-deep-q-learner) for a convolutional version using the SMM environment wrapper.

The code will be based on a package I've been working on while learning about RL myself. It's available here: [reinforcement-learning-keras](https://github.com/garethjns/reinforcement-learning-keras), and includes configs for solving other Gym enviroments including Cart-pole, Pong, Space invaders, Doom etc. It also handles running experiments with different agents, and some of the diffulties of working with Tensorflow. I've stripped a lot of that out here to try and keep the code simple. For those interested in theory, check the readme on github, there's a list of useful resources. There's an example using this package at the end of this notebook.

In [ ]:
# GFootball environment.
!pip install kaggle_environments
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev
!git clone -b v2.3 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib
!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.3.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

# Some helper code
!git clone https://github.com/garethjns/kaggle-football.git
!pip install reinforcement_learning_keras==0.6.0

In [ ]:
import collections
from typing import Union, Callable, List, Tuple, Iterable, Any
from dataclasses import dataclass

from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
import tensorflow as tf
import seaborn as sns
import gym
import gfootball
import sys

sns.set()

# Training keras models in a loop with eager execution on causes memory leaks and terrible performance.
tf.compat.v1.disable_eager_execution()

sys.path.append("/kaggle/working/kaggle-football/")

# Q-Learning

Q-learners are off-policy temporal difference learners, have a look [here](https://lilianweng.github.io/lil-log/2018/02/19/a-long-peek-into-reinforcement-learning.html#model-transition-and-reward) for details and context within the RL world.

## Components

 - Agent
   - Model, usually a neural network 
   - Replay buffer (the agent's memory)
   - Exploration method, such as episilon greedy
   

## Model

We'll use the GFootball enviroment with the Simple115StateWrapper which returns observations (state) as a (115,) array. This will be used as the input to a very simple neural network wtih 3 layers, which will output an estimated value for every possible action. The agent will either select the action with the highest value as estimated from this model, or randomly (chance depending on epsilon) select a random action.

To make learning more stable with deep-q learners, two copies of this model are used by the agent. 
  - "Action" model This copy is updated every training step and is used to estimate the value of actions for the current state. 
  - "Target" model" The copy is updated by copying the weights of the value model, this is done less frequently in order to keep the training target more stable.
  
The model is defined in this class, feel free to play with the arcitecture! Also note that unit_scale is set to make the model very small, this makes running in the notebook easier, but should be increased for real training.

In [ ]:
class DenseNN:

    def __init__(self, observation_shape: List[int], n_actions: int, 
                 output_activation: Union[None, str] = None,
                 unit_scale: int = 1, learning_rate: float = 0.0001, 
                 opt: str = 'Adam') -> None:
        """
        :param observation_shape: Tuple specifying input shape.
        :param n_actions: Int specifying number of outputs
        :param output_activation: Activation function for output. Eg. 
                                  None for value estimation (off-policy methods).
        :param unit_scale: Multiplier for all units in FC layers in network 
                           (not used here at the moment).
        :param opt: Keras optimiser to use. Should be string. 
                    This is to avoid storing TF/Keras objects here.
        :param learning_rate: Learning rate for optimiser.

        """
        self.observation_shape = observation_shape
        self.n_actions = n_actions
        self.unit_scale = unit_scale
        self.output_activation = output_activation
        self.learning_rate = learning_rate
        self.opt = opt

    def _model_architecture(self) -> Tuple[keras.layers.Layer, keras.layers.Layer]:
        
        frame_input = keras.layers.Input(name='input', shape=self.observation_shape)
        fc1 = keras.layers.Dense(int(self.observation_shape[0] / 1.5 * self.unit_scale), 
                                 name='fc1', activation='relu')(frame_input)
        fc2 = keras.layers.Dense(int(self.observation_shape[0] / 3 * self.unit_scale), 
                                 name='fc2', activation='relu')(fc1)
        fc3 = keras.layers.Dense(self.n_actions * 2, 
                                 name='fc3', activation='relu')(fc2)
        action_output = keras.layers.Dense(units=self.n_actions, name='output',
                                           activation=self.output_activation)(fc3)

        return frame_input, action_output

    def compile(self, model_name: str = 'model', 
                loss: Union[str, Callable] = 'mse') -> keras.Model:
        """
        Compile a copy of the model using the provided loss.

        :param model_name: Name of model
        :param loss: Model loss. Default 'mse'. Can be custom callable.
        """
        # Get optimiser
        if self.opt.lower() == 'adam':
            opt = keras.optimizers.Adam
        elif self.opt.lower() == 'rmsprop':
            opt = keras.optimizers.RMSprop
        else:
            raise ValueError(f"Invalid optimiser {self.opt}")

        state_input, action_output = self._model_architecture()
        model = keras.Model(inputs=[state_input], outputs=[action_output], 
                            name=model_name)
        model.compile(optimizer=opt(learning_rate=self.learning_rate), 
                      loss=loss)

        return model

    def plot(self, model_name: str = 'model') -> None:
        keras.utils.plot_model(self.compile(model_name), 
                               to_file=f"{model_name}.png", show_shapes=True)
        plt.show()


DenseNN(observation_shape=(115,), unit_scale=0.25, n_actions=19).plot()
Image(filename='model.png') 

## Replay buffer

The replay buffer records previous state, actions, and rewards. When the agent is updated, it samples a batch from the replay buffer to create the training set. These samples can come from different episodes if the buffer is long enough. This is necessary to reduce the correlation in the training data, which would be extremely high if, for example, the model was trained with just the last observation on each step. Neural networks don't do well with highly correlated training data.

In [ ]:
@dataclass
class ContinuousBuffer:
    buffer_size: int = 50

    def __post_init__(self) -> None:
        self._state_queue = collections.deque(maxlen=self.buffer_size)
        self._other_queue = collections.deque(maxlen=self.buffer_size)

        self.queue = collections.deque(maxlen=self.buffer_size)

    def __len__(self) -> int:
        return self.n if (self.n > 0) else 0

    @property
    def full(self) -> bool:
        return len(self._state_queue) == self.buffer_size

    @property
    def n(self) -> int:
        return len(self._state_queue) - 1

    def append(self, items: Tuple[Any, int, float, bool]) -> None:
        """
        :param items: Tuple containing (s, a, r, d).
        """
        self._state_queue.append(items[0])
        self._other_queue.append(items[1::])

    def get_batch(self, idxs: Iterable[int]) -> Tuple[List[np.ndarray], 
                                                      List[np.ndarray],
                                                      List[float], 
                                                      List[bool], 
                                                      List[np.ndarray]]:
        ss = [self._state_queue[i] for i in idxs]
        ss_ = [self._state_queue[i + 1] for i in idxs]

        ard = [self._other_queue[i] for i in idxs]
        aa = [a for (a, _, _) in ard]
        rr = [r for (_, r, _) in ard]
        dd = [d for (_, _, d) in ard]

        return ss, aa, rr, dd, ss_

    def sample_batch(self, n: int) -> Tuple[List[np.ndarray], 
                                            List[np.ndarray],
                                            List[float], 
                                            List[bool], 
                                            List[np.ndarray]]:
        if n > self.n:
            raise ValueError

        idxs = np.random.randint(0, self.n, n)
        return self.get_batch(idxs)

## Epsilon greedy

Epsilon greedy is used to force the agent to explore the environment. When the agent is asked for an action, it draws a random sample between 0-1. If this value is >epsilon, the agent simply selects a random action. Typically epsilon is set high at the beginning of training, and is slowly decayed over time. This class tracks it and handles the random action selection.

In [ ]:
@dataclass
class EpsilonGreedy:
    """
    Handles epsilon-greedy action selection, decay of epsilon during training.

    :param eps_initial: Initial epsilon value.
    :param decay: Decay rate in percent (should be positive to decay).
    :param decay_schedule: 'linear' or 'compound'.
    :param eps_min: The min value epsilon can fall to.
    :param state: Random state, used to pick between the greedy or random options.
    """
    eps_initial: float = 0.2
    decay: float = 0.0001
    decay_schedule: str = 'compound'
    eps_min: float = 0.01
    state = None
    
    def __post_init__(self) -> None:
        self._step: int = 0
        self.eps_current = self.eps_initial

        valid_decay = ('linear', 'compound')
        if self.decay_schedule.lower() not in valid_decay:
            raise ValueError(f"Invalid decay schedule {self.decay_schedule}. "
                             "Pick from {valid_decay}.")

        self._set_random_state()

    def _set_random_state(self) -> None:
        self._state = np.random.RandomState(self.state)

    def _linear_decay(self) -> float:
        return self.eps_current - self.decay

    def _compound_decay(self) -> float:
        return self.eps_current - self.eps_current * self.decay

    def _decay(self):
        new_eps = np.nan
        if self.decay_schedule.lower() == 'linear':
            new_eps = self._linear_decay()

        if self.decay_schedule.lower() == 'compound':
            new_eps = self._compound_decay()

        self._step += 1

        return max(self.eps_min, new_eps)

    def select(self, greedy_option: Callable, random_option: Callable,
               training: bool = False) -> Any:
        """
        Apply epsilon greedy selection.

        If training, decay epsilon, and return selected option. 
        If not training, just return greedy_option.

        Use of lambdas is to avoid unnecessarily picking between 
        two pre-computed options.

        :param greedy_option: Function to evaluate if random option 
                              is NOT picked.
        :param random_option: Function to evaluate if random option 
                              IS picked.
        :param training: Bool indicating if call is during training 
                         and to use epsilon greedy and decay.
        :return: Evaluated selected option.
        """
        if training:
            self.eps_current = self._decay()
            if self._state.random() < self.eps_current:
                return random_option()

        return greedy_option()

## Agent

The agent estimates actions values from states, and learns from its experience. Most of the magic happens in the .update_model, where the problem is formulated into something that can be learned by the DenseNN model.

In [ ]:
@dataclass
class DeepQAgent:
    replay_buffer: ContinuousBuffer
    eps: EpsilonGreedy
    model_architecture: DenseNN
    name: str = 'DQNAgent'
    double: bool = False
    noisy: bool = False
    gamma: float = 0.99
    replay_buffer_samples: int = 75
    final_reward: Union[float, None] = None

    def __post_init__(self) -> None:
        self._build_model()

    def _build_model(self) -> None:
        """
        Prepare two of the same model.

        The action model is used to pick actions and the target model
        is used to predict value of Q(s', a). Action model
        weights are updated on every buffer sample + training step. 
        The target model is never directly trained, but it's
        weights are updated to match the action model at the end of 
        each episode.
        """
        self._action_model = self.model_architecture.compile(
            model_name='action_model', loss='mse')
        self._target_model = self.model_architecture.compile(
            model_name='target_model', loss='mse')

    def transform(self, s: np.ndarray) -> np.ndarray:
        """Check input shape, add Row dimension if required."""

        if len(s.shape) < len(self._action_model.input.shape):
            s = np.expand_dims(s, 0)

        return s

    def update_experience(self, s: np.ndarray, a: int, 
                          r: float, d: bool) -> None:
        """
        First the most recent step is added to the buffer.

        Note that s' isn't saved because there's no need. 
        It'll be added next step. s' for any s is always index + 1 in
        the buffer.
        """

        # Add s, a, r, d to experience buffer
        self.replay_buffer.append((s, a, r, d))

    def update_model(self) -> None:
        """
        Sample a batch from the replay buffer, calculate targets using 
        target model, and train action model.

        If the buffer is below its minimum size, no training is done.

        If the buffer has reached its minimum size, a training batch 
        from the replay buffer and the action model is updated.

        This update samples random (s, a, r, s') sets from the buffer 
        and calculates the discounted reward for each set.
        The value of the actions at states s and s' are predicted from 
        the value model. The action model is updated using these value 
        predictions as the targets. The value of performed action is 
        updated with the discounted reward (using its value prediction 
        at s'). ie. x=s, y=[action value 1, action value 2].
        """

        # If buffer isn't full, don't train
        if not self.replay_buffer.full:
            return

        # Else sample batch from buffer
        ss, aa, rr, dd, ss_ = self.replay_buffer.sample_batch(
            self.replay_buffer_samples)

        # Calculate estimated S,A values for current states and next states. 
        # These are stacked together first to avoid making two separate 
        # predict calls (which is slow on GPU).
        ss = np.array(ss)
        ss_ = np.array(ss_)
        y_now_and_future = self._target_model.predict_on_batch(np.vstack((ss, ss_)))
        # Separate again
        y_now = y_now_and_future[0:self.replay_buffer_samples]
        y_future = y_now_and_future[self.replay_buffer_samples::]

        # Update rewards where not done with y_future predictions
        dd_mask = np.array(dd, dtype=bool).squeeze()
        rr = np.array(rr, dtype=float).squeeze()

        # Gather max action indexes and update relevant actions in y
        if self.double:
            # If using double dqn select best actions using the action model, 
            # but the value of those action using the
            # target model (already have in y_future). 
            y_future_action_model = self._action_model.predict_on_batch(ss_)
            selected_actions = np.argmax(y_future_action_model[~dd_mask, :], 
                                         axis=1)
        else:
            # If normal dqn select targets using target model, 
            # and value of those from target model too
            selected_actions = np.argmax(y_future[~dd_mask, :], 
                                         axis=1)

        # Update reward values with estimated values (where not done) 
        # and final rewards (where done)
        rr[~dd_mask] += y_future[~dd_mask, selected_actions]
        if self.final_reward is not None:
            # If self.final_reward is set, set done cases to this value. 
            # Else leave as observed reward.
            rr[dd_mask] = self.final_reward
        aa = np.array(aa, dtype=int)
        np.put_along_axis(y_now, aa.reshape(-1, 1), rr.reshape(-1, 1), axis=1)

        # Fit model with updated y_now values
        self._action_model.train_on_batch(ss, y_now)

    def get_best_action(self, s: np.ndarray) -> np.ndarray:
        """
        Get best action(s) from model - the one with the highest predicted value.
        
        :param s: A single or multiple rows of state observations.
        :return: The selected action.
        """
        preds = self._action_model.predict(self.transform(s))

        return np.argmax(preds)

    def get_action(self, s: np.ndarray, training: bool = False) -> int:
        """
        Get an action using get_best_action or epsilon greedy.

        Epsilon decays every time a random action is chosen.

        :param s: The raw state observation.
        :param training: Bool to indicate whether or not to use this 
                         experience to update the model. If False, just
                         returns best action.
        :return: The selected action.
        """
        action = self.eps.select(greedy_option=lambda: self.get_best_action(s),
                                 random_option=lambda: self.env.action_space.sample(),
                                 training=training)

        return action

    def update_target_model(self) -> None:
        """
        Update the value model with the weights of the action model 
        (which is updated each step).

        The value model is updated less often to aid stability.
        """
        self._target_model.set_weights(self._action_model.get_weights())

    def after_episode_update(self) -> None:
        """Value model synced with action model at the end of each episode."""
        self.update_target_model()

    def _discounted_reward(self, reward: float, 
                           estimated_future_action_rewards: np.ndarray) -> float:
        """
        Use this to define the discounted reward for unfinished episodes, 
        default is 1 step TD.
        """
        return reward + self.gamma * np.max(estimated_future_action_rewards)

    def _get_reward(self, reward: float, 
                    estimated_future_action_rewards: np.ndarray, 
                    done: bool) -> float:
        """
        Calculate discounted reward for a single step.

        :param reward: Last real reward.
        :param estimated_future_action_rewards: Estimated future values 
                                                of actions taken on next step.
        :param done: Flag indicating if this is the last step on an episode.
        :return: Reward.
        """

        if done:
            # If done, reward is just this step. Can finish because agent has won or lost.
            return self._final_reward(reward)
        else:
            # Otherwise, it's the reward plus the predicted max value of next action
            return self._discounted_reward(reward, 
                                           estimated_future_action_rewards)

# Play single episode

An episode refers to one play through an enviroment. It will take many episodes to train an agent

In [ ]:
def play_episode(env: gym.Env, agent, n_steps: int=10, pr: bool=False, 
                 training:bool=False) -> Tuple[List[float], List[int]]:
    
    episode_rewards = []
    episode_actions = []
    
    obs = env.reset()
    done = False
    for s in range(n_steps):
        if done:
            break

        # Select action
        action = agent.get_action(obs)
        episode_actions.append(action)

        # Take action
        prev_obs = obs
        obs, reward, done, info = env.step(action)
        episode_rewards.append(reward)

        # Update model
        if training:
            agent.update_experience(s=prev_obs, a=action, 
                                    r=reward, d=done)
            agent.update_model()

        if pr:
            print(f"Step {s}: Action taken {action}, " 
                  f"reward recieved {reward}")

        last_step = s
    
    if training:
        agent.after_episode_update()
                  
    return episode_rewards, episode_actions
                  

env = gym.make("GFootball-11_vs_11_kaggle-simple115v2-v0")
agent = DeepQAgent(
    # Realistically this should be more like 100,000!
    replay_buffer=ContinuousBuffer(buffer_size=1000),  
    eps=EpsilonGreedy(),
    model_architecture=DenseNN(observation_shape=env.observation_space.shape, 
                               # This is also set too low to make running in a notebook easier
                               unit_scale=0.25,
                               n_actions=env.action_space.n))

_ = play_episode(env, agent, n_steps=20, pr=True, training=True)


# Play multiple episodes

In [ ]:
def run_multiple_episodes(env: gym.Env, agent, n_episodes: int=10, 
                          n_steps: int=10, pr: bool=False, 
                          training:bool=False) -> List[float]:

    
    total_episode_rewards = []
    for ep in range(n_episodes):

        episode_rewards, _ = play_episode(env, agent, n_steps, 
                                          pr=False, training=True)

        total_episode_rewards.append(sum(episode_rewards))
        
        if pr:
            print(f"Episode {ep} finished after {len(episode_rewards)} "
                  f"steps, total reward: {sum(episode_rewards)}")

    
    return total_episode_rewards


env = gym.make("GFootball-11_vs_11_kaggle-simple115v2-v0")
agent = DeepQAgent(
    replay_buffer=ContinuousBuffer(buffer_size=1000),
    eps=EpsilonGreedy(eps_initial=0.7),
    model_architecture=DenseNN(observation_shape=env.observation_space.shape, 
                               unit_scale=0.25,
                               n_actions=env.action_space.n))

# Both n_episodes and n_steps need to 
reward_history = run_multiple_episodes(env, agent, n_episodes=10, 
                                       # Use 3000 for 90 minute games
                                       n_steps=3000,  
                                       training=True, pr=True)

plt.plot(reward_history)
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.show()

# Compare to a random agent

In [ ]:
class RandomModel:
    def __init__(self, n_actions: int):
        self.n_actions = n_actions

    def predict(self) -> int:
        return int(np.random.randint(0, self.n_actions, 1))


@dataclass
class RandomAgent:
    """A random agent that acts randomly and learns nothing."""
    n_actions: int
    name: str = 'RandomAgent'

    def __post_init__(self) -> None:
        self._build_model()

    def _build_model(self) -> None:
        """Set model function."""
        self.model = RandomModel(self.n_actions)

    def update_model(self, *args, **kwargs) -> None:
        """No model to update."""
        pass

    def get_action(self, s: Any, **kwargs) -> int:
        return self.model.predict()

In [ ]:
env = gym.make("GFootball-11_vs_11_kaggle-simple115v2-v0")
obs = env.reset()

random_agent = RandomAgent(n_actions=env.action_space.n)

ra_reward_history = run_multiple_episodes(env, agent, n_episodes=25, 
                                          n_steps=3000, training=False)
dqn_reward_history = run_multiple_episodes(env, agent, n_episodes=25, 
                                           n_steps=3000, training=False)

plt.plot(ra_reward_history, label='Random')
plt.plot(dqn_reward_history, label='DQN')
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.legend(title='Agent')
plt.show()

# Prepare submission

This is covered in a bit more detail in the next notebook: https://www.kaggle.com/garethjns/convolutional-deep-q-learner . In short, we need to create a single file containing the agent components we need for inference, including the serialsed model weights. We also need to add to agent function that acts given an environment observation.

## Get model weights

In [ ]:
import zlib
import pickle
weights_bytes = zlib.compress(pickle.dumps(agent._action_model.get_weights()))

## Create submission.py

Copy and paste the contents of weight_bytes into WEIGHTS_BYTES

In [ ]:
%%writefile submission.py

import zlib
import pickle
import tensorflow as tf

import numpy as np

from tensorflow import keras
from typing import List, Tuple, Union, Callable
from gfootball.env.wrappers import Simple115StateWrapper


WEIGHTS_BYTES = b'x\x9c\xad\x9ai<U\xef\xd7\xff\x0f\x91y\xc8\x1c*\x15R"\xa4\x12{\xadN\x8a\x84\x90T\x9a5~5S*\x95\xca\x1c\x92\x90\xa4\x92P$!!){\xad$M*i\x96&\xcd\xd2\xa4Y\xf3\xdf\xf7w\xdf\xaf\xdf\xe3\xff\x83\xfbzp\x1e\xec\xb3\xf7u\xad}\xceZ\x9f\xf5\xfe\\\xaf+\xbc\xcb\x8c`\x89\xc5\xdce\xab\x96\x06\xad\xb5\x9e\xbb|\xc5|\xeb\xa5\xab\x96\x84,\x9c\xbdb\xc5\xec\xb5\xca\xb3V\xcc\x9f\xbb|\xd9\xca\x90\x15\xab\xe6\x86(\x07\xcb\xfc\xcf]\xca\xcb\xe6\xfd\xcf\xb7\xc1\xb2\x1e\x92\x98\xe0.\xa3d\xe6\x04\xcbm\x0e\x96\xf7\r\xeej\xe1!\xe3\xb1\xd2C;6X\xe1\x7fo\x9d\x17\xb26h\xber\xb0\xa2\xbf\xacD"Y`\x1f\xac\xe4!\xf1\x90\xd9\x1c\xac\xec\x1b\xacb\xe1\xd1\xc5_\xa6\xf3\xaa\x10\xac\xea\xe5\xe5\xe5\xfe\xb7s\xfc\xe7\xc3C\x12\x12\xac6\'\xde\xd9\xb4\x8fD\xb2Ny\x1f\x14.K\xa1iw\xc6\xb0$`\xff\xf1j_g\xfe\xf8\xa7\x85\x8e\x1bhs\x88\xebl\x98\x1b.\xcbm\xb1\x99\xe0\xf00K,\xdb\xf9\x9b\x86\xd4M\xc4=c\xba\xe1\x81\x9dsI\xb2w\xb9\xa38\xd8\x89\x14t\xefQ\xf8^c*\x0b,\xa4\xcf\x1f\xd2\xe9\xf2\x88\xd9\xc0O\xd5y\xe5\xc7\x11\xf8j\x91"\xfa\xdb\x92\x98\xbe\xce\x89\xad\xa6\x18\xe2O?->m\xed\x81\x95\xbf/@\xe9\xad\xb7\xa2\xe3\xd2\xfb\x90\xf5\xe6.8\x86\x9e\x87\xe9\r3a\xe0{g8\xdbW\x0eU/\xf6e\xc9\xfa9\x8e\x06\x7f\xfdI\xaa\xf3\xa3\xda\xbc\xef1\xba\xaa$\x8b\xcdW_\x81a\x97\xc7\xe0\xaf\x96F\x81\xe7\xeaE\xd9\x12E\x0e\xd7p\x13\xd3\xc2\xd5)c\xa8\x12\xebM\x1c\xcc\xf17\'\xa0\xac\x957{\xce\x92\x85\xba&5\xce\xb7\xdc\t.7dqEI\x01\x18\xcb\x99\xf2@\xb7!\xa0\xd3\xdf\x1eV\xfc\xbc\x0c\xfe{\x0b\x85\x07\xe1C\xf8\xd7\xf9O\xf4\xe2\xe0\x15h\x1d\xdf\x05\xe6U\xf6$\x1f\xa37b\xa3\xab\x99\xe0\xf6;N\xf0YrN\x9c?H\x03K\x87)\x89\x8bV\xbf\x87\xd4\x87[\xe0N\x0fc|\xf6a \xa5[=\x05}\xdb.\x9c\xf4\xe3\x16\x84\xafl;\xb9ff\x7f|\xf7\xfb)\x9d\x0c0\xe5\xc5jf\xd8w\xf0\x10\xd4\x97ugu\xd3\x9b\xb07\xca\x8c[\x0e\xf4\xe5\xcb%\xc1 \xf1]rr\xd2\xf9\xaf\xa0\xd8\xfd,\xd9\xdd}\x05/\xc6\x98pS\x9d\x13\x9fr\x18\xc3#\x12D\x12\xa3\xdc8s\xdeU23V\xc6\xe6^M\xe0\xd3\xab\x1b]\xbdc\x80\xb8K\x95\x07\xedq\xe3Y\xdev\xbc\xf2\x1f\x1f|`1\x96\r\r\xb3@6b7<\xfc\x93\x0bm\x1e\xd5\xf0.\xe4(e)\xc5\xd0\xb3\x9f\xdap\xc7h\x0cj\x0cpb\x9f5\x9bD\xbd\x80C\xf0U.\x83\x9eI\x86A\xf1\x19\t\xf6\xfc\xfe\x19\n\xbde\xb8&<W\xf8\xba\xf1#\xa4\xac\x16X\xb6p3\xd8\xbf1\xe2\xcf\x0fl\xc1\xe3\xde \xf6\x9b3\x9e\xf4\xd6\xe7\xc0\x803\x97\xc0\xc4p*\xd5\xbek!\x89\xd6\xb7\x13\x89\xa6\x8b\xc8\xb8R\x96\xd2\x1e\x08\x10>\xd2\xd9)\xc3\xb6\x80P!\x8b\xea[\xf7Bq\xd6\x18|\xd7#\x07\xfa~\x1b\x86\x8e\n_\xc1\xab\xb1\x19\x82\xacU\xf8\x1fmk\x14\x15\x86\xc203mlAOA\x12o\xc8\xf5F\x0c>\x8b\xfc\xc5\xed\x03\x94X\xd30\x89Z\n\x17\xd1\xf5\xb0\x07P\x15\xb3\x9f\x9al\'s`\xd63r\x9c\x9c\x0f\x07,\xfasO\xbdh\xa80R\xa2\xa0\xd7\xfa\xa0\xab\xf3\x94rVwe\x93\xfc*\xa7\'\x83\x15Y\x96\xea\xc0\x7f\xe9J\x1a\xb9A\x9e;*K\x05\xb9\x81\x13\xe1A\xbe\x01Z\xae\xb0\xe7\r\xb1\'i@\xc5%x0s2\x86_:+\xd8\x16y`\xcd\x93A\xa2l}\x04\xdd\x7f\xd8\x8f\xfdt\xe7\x03\x1b(p\xa9\\\x02\xa9\xdd\xee\x8e\xb6\xf7Gc\xaf4Y\xec\xd7\xde\x9bw\x8c\xd6D\xe5\xed\xd5p/\xd4\x9a[Lv;\x8dz\xf2\x88\xb4\xdc\xee\x80V\x83+[\xaf}\x0f\xf7\x97\xa4S\xf3\x1f\x156|Y\x06\x0e7>\x0b5*\xc5\xc2\x9e"\xa6\xb8\xc8\x1f\xc2\x05\xd9\x91\xe4\xbdB\x85\xe7\xb7^\xa6\x08\xbd\xde\\un\x1e\x8dP\x17i\x91\xb6<\xdf\xbf`\tC\x94\xcd\xd9e\xec@\xac\xad\xbfB\xd2\x11\xd3(p\xa6;\xf5Kq\xe6\xb5\xd3\r\xf9kq4\xad\x9ez\x1cD\xd8D\xb7\x1f\xfd\x81\xb8\xfd*|{c\x19%\xae\x08\xa6\x15\xeb\xeeAL\x8c\x04\xc7i\xb8\xe0\xaf\x07:\x94yQ\x0ejC/\x82\xd5V=t\xfe\x99L\xf3\xc6\x97\xd2\xa3\xbe>\xdc%\xa9\x9eJl{\xf0\xdc\xae\xee(\xeeO\xa2&\x9bH1\xbc\x8a\xaa\xcd\x16[c\xf7-C0\xff\xc6\x1609\xaa\x01\x9f\x7f{r\xfd\xc4&\x1a\x9ec\xc2\xef\x86\xb6\xd0\xd8\xf6\xafT\x93\x7fMT\xd5+&\xdd0O\xbe\xbc\xdb\x16\x14\x03\x13(\xe8\xe3GQ2\xde\xadZlZ\x08\x03{\r\x07\x93A7\xab\xa7~)\x00\xb7\xc8\xf1\xa4\x9c\xd3\x08\xfa\xaf\xba!\rQ\xe1\x95\x9a\xc3\xf0u\xe7{+\x1e?B\x83\xafZ`\xf5Ym\xf4\\?\x84\xf5\xea\\xo\xe3\x18\xae\x19\xf3H\xf8\xf3%\x02\xe4\x8c\xe4\xc9\xeeW4\xa8\x04\x00\x06U\x86Q\x88\xaa\x07l\xfb5\x1c}\xce\xcf\xa5\x96\xe7\'\xc4\x81I\x8e0\xb9W\x1f<\xbbr8^-UEIR\xc7p\xc33;\xa9\xb4h\x10\xaf\xfb\xeb\xcb\xbdt\x8d\xd1\xbf\xa4\x1f-\xc3\x9e|~\xcf\x0f*Q\xd6fW\x9b2@\xdb\xc3\xd0/U\x0e\x078\xb8\xb2\xc3\xf2\\\xaa\xbaS$leGT\xe4uT\xba\xf6\xab\x90\xf4\xcf7\xa8\xef}\x1f\x1cKO\x82\xcf\xe0\xd7b\x8f\x06}\xbc8\xea\x0b\xb5\xcf1\x13[\xbb(Ak\xccsA\x92v\xc3\xf1\xa1F;\xcc\x1ec\x80S\x82\xe5\xb0\xf4\x8a\x1f\x8d\xda9\x1a\x93\x8d]\x05?7\x0bp7\xfe\x03\x9f\xc6\xd6\xd0qU\x1f\x94\xd87;\x8e\xcb\xb0D\xf3yg\xc1`\xc3~A\xf9R\x19\x04\xc9\xe4\x91\xb9\xce\x16(@mJ\x98z\x14\x9a\x82\x13h\xa3\xfc9\xb8\x15\xe7\x89)%\x12\x8c\xdbfG\xd6\xf1\xbe\xfc\xa7J\x96\xd7\xd8}\xa7U\xe7dQWo2{e\x1f\xa2\xcb\xa3\xf6\x91s\xca\x05x9C\x05G)\xca\xf0\xee\xee\xf9\xd0^<\x86\xd1|\x1b\xc5\x1b\x8c\xc5Fc9\xa1:\xe2\x03Ig\xac\x13\xd3\xbe&\x91Y\xc27Zo\xf4\x93\x9e\xd5\xcc\x83\x90\xe2O\xd4M\xd1\x8d\x9b\x8a\xde\x8aUc\x9a\xa9\xed\xbc;/{\xf6\x1c\x14\x1e\x1b\xe0\x93{;\xc8$\xe62\x99l<)D\xfaFA`\xa9\x84\xef\xa7*\x82\xf4\x8ab\xb5\x83\xf4\xbc\xb0\xd1\xf4\x05H\xdb\xdd\xa9\xa0d\xb7\xb0~\x97\nJ\x86\xcf\xae\xf6\xedUJ\xfao\x06\xe3\x0cQ\x1e\x9b\xea\x06Q\x9cjo.\xedfK\x05m\xe3hU\xfb$N\t\xe8\xc3\xbb\x13\xf2i\xc0\x8f#\x90:\xff(\xd8NWB\x0b\xdd\x83\xe2\xb25\x87 \xbbe\x97\xd3\x12\xc1\x89B\x0c\x07\x81\xd4\xbf\xb7S\xf6\x92H*\x1d\xb2\\T\xfe4\x80\xc3\xb5\xb3\xc4&\x85c\xc2\xaa\xc7\xb7(\xd4:[4\x18\xae\xcf\xb7\x1f\xa4@`\xba\x1f7\xb9\xbc\x12r\xcd\xf7\x83^I\nD\xa49\xe2\xf6f\xe5N\xad\x9dF\xd9\xc6QN\xa1\xb5\x9b\xa8u\x89:t,\xdcHe\xd7\xb7\x90\xf4c\xa2X\xbc\xd9\x10=\x13\xac@\xe2x\xceq\xccF/D\xb1\x05\xec\xee~\x06\xed\xd2\xc9\x18j\x9e&\x1ejB~\xd6\xd7\x82N\x15N@\xeb%\x0f@C\xcb\x19\xad\xe2l\xb0\xf9o\r\xb4\xec:\xe1\x14\xae\xe3$d\xeeXA#\xf2\xcc\xf8P\x80.*.:L\x9fP\x9b\xfdv\xf6`\xbc\x90A\xea\x96\xb7h\xf7\xc8R\x08\xfa\xa6\xc1\xd7B=qu\x9fo$\xdcP%\x83\xd0\xf3bcC\xbb i\xdbB]\x86+`\xdb\xe5[p,\xbf\x176D\x15\x10\x8d\xbbG\x87g\xd8c\xd7Y"\xadn\xdfI\xb3\xd3mpu\xc1>\xb0Yw\x91.\x16\x97Q\xfa\xe8\xa3P\xec\xe1\x8e\x8d\x11[\xc8l\xf0X\xdc\xe2?\x0c\xbd\xa2: ;\xad\x8d\x1a\xa6\x1f\x86I\xb1\x7f\xa1\xdc\xc9\x1dM\x8c\xaf\t\xdb\xd4l1Y]O\xb0t\xf4\xc0E\x06?\xe9\xec]+^\xb6\xa4\x83\xa6n3b\xbf\x1f\xf7H\xfe\x89#\xfa\xff,\x11\xa6\xfd\xfeE\xed%\x9f\xc5C\n#q\x81\xf2G\xcaX~Q\x9cp\xb1\x0b\xbb\xbd\xae\x10\xa1x8f\xdf\xe9!*\xac\x1c\xc9\x83\x94\xacpskwV4uf\xa9\xcb\x1c\xf1\\L:\x18F\x94\x93\x83\xb9\x06x\x9a\xb8\xc2\x9a\xbc\x1a\xb2\x98\x15+\xbc9\'\xcf\t\xb7w\x91O\xe5}\xa1&\xe3\xb6\x90\xbc@\x85w\xbd\xd1\xe5\x9aY*bX\xdcB\x98e>\x081\xe05\xe8\xb8\x15\x92\x94\x8eUo\xad\x96\xf2_s\x17^\xe6\x9f\x0f\xc3\xa2\x07\xa2\xf1\xe4 \xc8\xc5\x9f\xe4\xc5[\xa1V\xf4\xe3\xef\xf7\xfb\xa0\xf6.\x1b\xae\xb0\xee\x05\x05{\x8e\xd2\xf1\x0bZ\x88\x92^\\\xa9\xb7\x05N\x8fP\xc7\x86\xfd\x93\xb9\xc7\xfb7\xd4w\xc1\x00l]\xa1\xce\xbd\x0eL@\xa3\'.\x18e\xac\x8b\xa3Z\x1e\xd2yC{\xccL\x1c\xc2\x03\x9f\xfd\xa09\xa5\xaf@Ug\x10\xfcIx\x01On\xa4B\xcfYO\xc1\xb4b\x00\xcflN\x01\xc9\x9ca\'\xed\x0b\x00.\x8c\xfa+\xdc\x7f\xed\x0cR\x8dL\'\xe3\t\x83A\xb3\xbc\x1b\xdfQ4\xe1y/\x06\x81\xd7\xf7\x8fp{\xf6\x03\xc8\x1e\x14\xed\x14\x1b\x92G\xd7\xb5]8a\xd4\x19\xb0.(%UXC\xf30\x93\xf2\x13\xe3\xa1\xbb\x95\x1e\x9ahJ Hv\x13\x9dO\xb6\xc4)\xcf\xea\xc9\xa6o\x04\x95\x94\x97\xc0\x89\xf5\xdd\xb1>*\x9d6?4G\x85D7\xce\xed\x7f\x01N\x9eN\xa0\xbaX#z{w\x18\x8a>\x11d\xafx\x8a\xb6\xec\xd2\xc7\xec\xea\x9b\xd5\x97\xb5WS\xe6V]\x88+\xfaF\xc9\x9a\xeb(<\xda\xc9I:c8\xcf\\\xfd\x9b\xee\xa7\xe4\x11\x7f\x1e\xc5ngU\xb8\xdf\xad1\xe8\xb5\xaf\x06\xee\x1e\xe8\x87\xd3\xe2\x1d\xf8\x84\x8e\x0b\xaa\x1d\xef\x8f\xb9\x9e\x89T\x178\n\x1a\xc4wT\'o\xc1\xfd"M0\xd8r,\xde\x9f#O\x0bM}P\xb7\xbf\x1f\xb6.\xdd&D\xce4\xe7+\x83\x94q\xf6\x05\x03\xbc<\xd2\x13\x96\x1b\x9a\xa3E\xe3a!])\x95\xea\xde.\x85\xc5\x96\xe6(i\xdbJE9nP\xe22\x8c\x1b\x0e\x97\x91\xec\xe1\x1c*\xbahG\xeb\xe4\xce\xc0\xeb\x17\xce<6x\x12/\x18\xea\xc4\x8es~\xc0M\xcf\x85d\xf1\xf8\xbe\xd80)\x9c2\xa2\'\xd0\xd7\x1d\xe1P\xd3\xb8\xc3\xa9\xf6\xa41\xaf\xbc\xeb\x81\xb7\x07y\xf0Ja4\xcb[x\xe2g%?\x10\xa4C\xa0Bg\x1a}m\xbf\x01?\xaa~\xc0\xce}:\x18xi\xb3\x98\xf5\xf1!\x84\x89\xd3\xa1\xe9K\xbex\xeei\t\xc8}Qa\xdb\x89R\xec\xdf\xd8J\x81\xba\xfdH\xf6\xec \x96l=\xe8\xd8\xa4\xb3\x83zz\'BO\xafTz\xf3I\t\x8a"\xbcIG\xa6\x8e\xdcN\xef\x14N\xfe\x93\x0b\xea\xc9\xb6<u\x9f;\x87V\xe7\ns\x1a\x9d9H\xd9Zt\xcd\xef\xe4<\x9b(XQ\x02,\xbf@\x1bK\xa3\xbc\xa9\xfd\x9d\x0b\xbf\xdb\xf3\x1d^\x0c\x1f\xc6\xfa\x8e\xbe\xbc\xa9@\x937\xef\xbdD/?\xa8\xa0\xbf\xf1^\xe1U\x81>\xc6\x9d\x1bL\xc3\x1fx\xa2`\xbd\x10\xd4\xc76@\xb1\xa3/&\x16,\x87\xd2\xe5A\xe2\xdf6}\xecY\xec\xc1\xcb2\x93a\xcb\x8a\xb1\\kr\x13Z\x9d\xf3\x85\x109-\x0e\x1f*#\\\xeb\xdd\x8fC\xfe\xdaA\x98\x91\x07\x84\xce\x7f\xda\xa9\xdb&`\xa6\xe7\x8d\x8f\xccn\x911;@N\xefa|y\xc2^j\x19xJ|q\xb8\x1d\x14\xcbk\x05\xc7\xae\xda\x9c\\\xeeO\x93~6\x90fC\x9b(\t\xfc\xe3(I:pRZ\xd0S\x08\x98\xf0\xa1\xbaq\x17rU\xecS\xea(\xdfI\xdd\xd3\x14Q3\xd6\x9b,\x8a\x1d\xa9)\xb2\x98\xfc\xbf\'\x88\x05\xcfL\xb8\xb90\x03\xbe\x01rK\xccYz\xbcM\x0f\x1b>\x1d\xea\xe4\xb0\x00\xf0\xd7\xcb\xea\xe4\xb3\xab N4\xa4S=\xba\xe2D\xe9#\xca\xa6!\xb4\xb0P\rG\xdc\xb4\xc5\xd4\xebohEF\x04\x14IW\xc1\x865\xa6\xd82l\x04\r\xc9\xf7\xc1\x1b\xdf-0\xd0\xf9\x87\xd0\xd5\xd1\x92\xe5\x06\xc4\xd0\xf3\x05\x9e\x18[|\x90\x1a^\x9d\x81\x90\x936\x94+\xec\x81\xcb\xee3\xc8\'b\x8fX\xfce\x00\xe6\xdf\xc9\x85\xba\xce\x9a1\r\x91\xe1\x1d\xe8\xc9O\xb4\x99\x96\x0e\xf3\xc7n\xdb\xf5pB\x86\x03\xcaG5\xd1\xc1rK\xfcQ\xf3\x10\x06\x8e\xea\x06\x9a_=\x84Q\x97\xddqAF8\xb4l\x8a\xa1\x82\xe9\x99bi\xda0\x1e\xb4\xe8"\xa9\x99\x0f\xc0\xafO\x0e\xc1\xd4o\xf2\xdc\xf6\xea\x0fe\xae\x1cB\x13\xf5\xadxQ`\x0e\x18\x189\xb3\xdc@M\x8auN\x807?\x0c\xe0\x82\x89+\xe7L\xe8\x85r\xf3\xd6\xc0\xabM\xa3qe\xf5`\xdc\xba\xce\n\xe7\xfe\x9d\x845\xe5\xcf\xa8eByu\xe9\xac\x9e\xd4\xab\x9f%+F)Ra\xc49\xb0I\xe9.\xbe\xda\xdf\x85\x8b\xe2Fp\xcdDsJ\x9e\xbbA8c\xa5\x82\xca\x9f\xcb(.\xc4\x90ed\xdc\xf0W\xac/Dk\x8d\xe3\xa2\xa7Zph\xf4\r\xf2\xf2r\xe0\xb9\x12e\x9e\x9a\x92\x0c~U\xb9\xf4\xce-\x81\x06\xe6\x0f\xa5\xe5\xbf\x1d1\xf1g\x0f\x8a\xdd\x9dO7\xbd\x13\xa8\xd5Q\x91\x7f~\xb3\xe3\xfe\x8e\xa6h\xbfq\x15D\xaa\xec\x04\xcb\x11}\xf0s\xe2R\xc8\xf5\xcc\x81|SEv\xba#\xc5\x80U\xbfh\x82\xfaC\x92LP\x05\x9b\xa4\xdd\xe2\xec\x9a\x89\x98|\xf8\x9008\xc6\x04\xc7\x16\xeabK\'\'\x85Gm\x14\x8a}5\x10\xbb\x15\x82\xc3\x19M8w\xbd\x16l\xbfW\x9346W|py\x10\xb6\xfa~\x13\xfbD+0Z2T$\xa9s\xf2\xe0X\xb1\xc6\xe9\xa9\x90p\xbd77[n\'#cG\xec\xeaX\x05\xd15\x03`\xd9\xbaK\xd0\x16_\x0f).\x7fi\x8e\xef\x11\x88\x9b\xaf\x02\xd9\x8d)N\x97^\xe8\xb3\xe2\x05\x0b\xce\x12\xb6\xd1\xba\xc2\x01\xec1\xc9\x12\xdd;\xfb\xec\x91\xebe4\xfb\x8c\x11\xa6]\xef\xc5-rUNFK\xdd\xf8\xc2\xfe5\xd45\xf7!}\x88\xb2E=\xc5\x18\xb0\x9b\xaf\xcb\x9b\xac\x86\xf3\xeb\x05\xfe\x18\xb8\x81\xc4\x0bJ\xd1\xb4l"\x03y\xdar\xd0\xc5E4\xfa\x8b3lu5aiH\xdf\xea\xec\x1b]x\xf2\xd2\x1eX}\xd9\x00M\xcc\xc2E\xadg\x9d}\xcdl\x92p\xea\xa67\xca\x97\xfa\xb2\xc2}y\xbe\xb9k"\x15\xbf\x97\xe1F\xedE\xe2\xaf\x9c\x01\xd0\xabz\x08\x97O+\xa7\xb0\x08%\xd2\x18\xe9\xcd\xdf\xae\xba\xe1\x93\x91\x87\xe0\xcf\xa3R8\xe9\xeb\xcd\xb2\x0e\xfe\xac\x15\x99\x03\x01\x9a\x99\xc2vCU\xae\x8cK\x05\xfbM\x96\xbc\xc47\x00\n\x0f\xc4\xc3$\xdc\x01\x93?\x8c\xc7\xeck\x9ab\xce\xbaF\xb2\x82>\xd8\xf5\xfeW0\xae\x1c\xca\x87\x7f\xf6\xc4\x15\x1b*![CA\xecs\xe86\xdc<\xb5\t6\xbfr\xc1\xcc\xf9\xfdx\xd3\te41\x9a#Xm\x9c\x84E\xd7\xf4\xd9\xb4\x87\x1e\xc5\xd5\xbc\xa1\xe1\xcf\xec\x11+\xf3\xe9\xd9\xdba\xf4\xb5\xe9\x17\xb5\xdc*\x11\xd2\x8d_@N\xa3\x02\xce{\xe5\x0b5\x97f:\xb9\x07\x97\x83\xfc\x97\xc9<\xb4f;\xd8L\xee#\xc4\xcb\xbbbd\xdf\x12X1\xeb0(\xf5\x92\xc5h{K\xa8\xd1\x9b%v\xb8\x9d\x16\xaf\xca\xab`\xed\x88\xad0\xabZ\x1b\xd3\xb5\xf2\xc0\xbaO"\xe9fy\xe2\xeb\xc5\xc3\xd9\xb6d \x8f6\x99\xc8_3\x1f\xd3\x8a\\w\x8eR\x1a\x88M\xfe\xc6d\xde;\x19\xce\xf4\xed\x87}\xd6\x9d\'\x873QB\xd7%]\xb9\xd4\xee\x0c\xb9t\xf5\xc2\xc7..\xd83;\x13T\xc7Z\xf1x\x89<\xab\x1a\xad\xa6\x15M\x06|\xf9\xca\x14\xd8\x98\x91DQq\xf7h\xc3=\xa2\xc0\xbdqT9\xd6\x99%\x051\xc3\x7f}\xac\xa0Y\x9eg\xe9\x87M/\xae^\xd0\x15\xed\xa7\x99\xc1\x8c7z(\xb7M\x83\xf2\x9e\x9a\xa2\xa49\xffD{g\rx7\x1b\xe3a\'gT\x1e\x1eO>~\x07\x05\x13\x89/W\xd5\xcaQ\xb8\x96\x94^t\x1f\xc0\xef\xc3l\xf9\xf2E\x01f\xbbN\xe2t\xdf\xdb\xf0B\xed+\xc4$\xf6C\x9d\xad\xd6\x1cph\x87\xd3G\x95\xdb\xd0\xa3E\x17\x1f&\xfd\x80*\xd9\x0ea\xd2\xcc+t;7\x95\xf4\xf4j\xc1/\xb6\xd3\xbb>4\xe4\xda\xd2CP\xf8r3\xb5<Nq*\r0\xe3x\xefn\xe8\xdf\xe9\x91\xfa]\x90\xe2\xc1\x8e\xab\xf4y\xfcd\xb8\xe9\xefD\x7f\xbd,P"\xd5\xa8\xda\x92\xa6\x83\xb6Z#\xd1w\x11\xd3\xd6\x0f\x13\xd8\x84\\\xc5\x94\xaa\x1b\xd4(\x19\xc9\x99;zsS\x88\x0c)\xb7\xec\xa1\xcdY\xbe\xe8\x1f\x11KKS\xfd\xf0\xf4A-\x9c\x9f%\x8b\xcem\x87\xc8[G\x86\x0b?\xbe\x83\xef#\x06\xe0\x8c\x18m\xbc\xd9\xe0\x0bi\xeb\x86\xd2\x01\xafN\xc6\x0b\xa8\x83\xe6_C9q\xe8lHK\x98\x03\x1b\x1e\x95\xd1\xa9\x80k\x94X\xd6\x83\x87\xe8\x8f\xc6m\xeb\x1c\xb9=\xa3M\x9cz\xe3\x17\x9c\x1e\xe9\x8esO\xbb\xa2d\xd8\xbd\xea\xc4\xf2n\xc4\x8d\x0c\x12%#\xf1k\xc0aP]0\x91\x92\xba\xab\xf3\xde\t\x88AZw\xc4u#\xdf\x83}\xddL8\xf7>\x07\xc2\'\xbcu\x0cP\x1bM\x9a\xea~b\xcd\xce%N\xd3\xd5\xed\xf9\xe4\xdc?Pv\xea\x11\x1c\xbd\xd7\x07\x9b\x93>\xd0\x18%c\xf4\xe9XD#}\x0c1r\xe9\x03\xd0\xea\xd1@i\xea\xa6\xb0DXGa8\x06J\x16F@t\xf3p\x08TN\xa39/\xed\xf9y\xb0\x11~y\xa2\x8f\xaa\xa748|\x97\x16t|\xd6b\xcd\x1a\x13\xc1/e6\xd4j\x95\xc1\x97>\x86<a\x94*:\xbae\xd1\xee\x1d)\x14\xd7\xa3\x99\x9e\xf5\xb0&{\xe7\x91\xd0\x9a?\x9f~Mo"\x93\xfc\xd9B\xf6\xec\x99B\x1f;3\xfe\xe2~\x9c\x02\xb3\xf5\xf9\xe2\x15y\xac\x9b\xef\x06\x99\xe6\xf2\x10f\xbb\x10\xb4\xden\x81G\xfd\xddq\x91\xdef2\x1f\xf7\rb\x0e\xca\xa2\x199\xe0\x12\x83\x08*\xe2\x11@\xd3z\xf3\xcb\x1d\xbaxq\xc80\xde\xefj\x81\xfd\x9fj\xe0\xe6Xy\xac7w\xe0\x03\xf7kh\x8e\xeci\xb08S)T\xec\x04\xf2=\xf2\x18\xfe\x0e\xb8D\x89\x0e\xa3\x99N\x18\xe1\xf4\x17\x8ep\xbe\xd4\x05M*\xf6;\x85\xbf\x1a.\x94\\\x1f\xc0y\x03\xcd1\xab\xe1\x0f\x8d9l\xd7\xe9A\xdciF\xd7\x81\xf8f\xd4h\xee\xe8\x9f)4\xe5\xc8\xc2\x9c\xb7)\xb0v\xc2\x10\\\xb0\xe7\x01i\xac\x9d\xc0\xe3\x0e\x1a\xf1\x07\x1fg,8{]\xf4\x8d\x13\xc1+\xfd<|\x999\x1c\x0f\x96\xd9\xa3\xe3\x02W6?\x96\x07Y\xcd\xe3x\xb9\xbd)\x9a\x9c\x88\xa8\xfe\x13\xfe\x1e*\xd6w\xea\xf5\xfdb\xc8\x98\x1d!\xb6\xc6v2\xed\xa6\x19N\x01?w\x08*\xcd\xc3Xb<\xfd\xa4\xc9\xda:\xc1\xfa\x9e>\x87z\x8f\xe51\xb9\xdd\xf1O\xd3f \x175T\x15\x94\xf9\xd5ug\x9e~\xb1\x96\xfe\xf9n\x8c\xa7\x1e\x0cC}w\x0f\x0e\x0f+t\x0c\xd8)Kqm\xad\xe2\x12\xd3^\xf4d\xe7#8\xb0 \x9d\x8c\x8c\x86\xe0\x9f\xef\'\xa1\xc9\xd5\x9b\xf4|\x0f\xc1\xf6=\x13\xd8o\xa9\x00w\x15\x1dP\x12\xf0\xe7dE\x9e)\xf9\xdd=OC\x9aU9\xb9\xb4\x17\x99\x0c\xd5\x12Z\xfa\xa4\x89\tE;)\x87=\xb0mx:\x85H\xb4\xe1\xcd\xe4Q4\xb4&\x1c,<;s\xff\x80\xa5\x90\xa9;\x06\x8e=,\xa7\xfe\x13t19GE\x8c\x8c\xaa\xef\xd4\xfd\xb7\xd0\xe3Q\xa77~\xf8\x8eJ~V\x834\xee\x9cp\xb2\xa5\x03Z6[\x89Iu\xbfi^\xb7I\xe4k]D#\xbe\xb9\xa2\xb4\xffj\x81\xf3~\xc0\x8cO\x93p\x99\xcbc\xc8\x8d\xaa\x85\x15\xc1C\xf9\xd7X\x1b\xb0\xafYJ5\xe3\xbb\x82\xe6\xbcv\xf1\xc6\x90\xfex4\xe0=\xb5\xa5\xda\xf3\xc3g\x9b\xe1\x86\x967:\xaa~\x84\xcaDmN\x04G\xb8\xffg\n\xf8?\x1b\xc6\xa5\xad\x83\xb8\xed\xe2\x07\xf84\xb5\x89>n\xae\x84U\xeer\xd8{\x86\x1b\x7f<3\x96\'f\x8f\xc3\xec\xe7\x11\xc4\x1b\x8f\x82\xbe\x9b9\xa3\xa36\xf7\xd4z\x01\x0e\x97\xba\xb0\xb8f\x14\x07M\xd5\x10\xc2m\x12\x85Q\xaa\x03\xb1H\xc5\x19\x1a\xc6\xd4P\xfb"e\xd8\xbe\xb0\x83$\xef\x8f\x08\x87\xeam\xd1d\xcf%\xa7f\xbdX\xc8\x9d\xe7\xc9\x93\xf2\xea!H\xa5\x92~u\xf4\x83e\x8d\x07@\xb1\xe8\x9aX\xe8q\x0c\x14\xc7+Q\x97\xc3~|@\xbf7\x84\x9d\xb0\x86\xd2\xd6YtY\xb7\x81\xceY\xbd%\xadnz\xdc\xb4P\x8d\x92W\x82\xa0y\xd5F\x18-X\xf0\xb9\xde\x99P\xb9R\xca\x15\xfb,!A6\x01|N\r\x16\xb2\x0f\x92Xq*\xac\x93o\x86\xd2\x85\xf7\xbfD\xfbT\x19\xdav\xa7\x1f6>\xdcB\xca}\xef\x834\xec\x8aXS\x1a-\xca\x98O\xc6?\xe7\xf7\x81\xd7\xbao\x9d\xef1\x9c%\xc5k\x9cJ\xef\xce\'\xa3C\xb2\xb8\xe2\xc7\x13\xa81\x98\xc0\x89\xfb$`\xf5\t\xf0\xe5Z[\xec\xf07f\x7f\xdeG:\x9aN\xb0\xc8\xfe\tH\x82\xcfWWJ\xcc\xd8\xab\xb6\x16x\xdbm*\xec\x17CEv\xe6p\xe2\xdd \xd4\xcc?*\x1e\x7f\xd3\x97}\xaf\xbe\x80\x02\xd72:\xadx\x97\xd2/\xa5@TuO|s\xd3\x07\x0ex\xc5\xd1\xed\xb6\xe7 c\xe6\x82\xdf\xd3m\xd1s\x9b\x1e\xfdj\xdf\x08\xa7\xe3\xc7\xa2\xd4KV,\x97\x05\xaeI\x18\xc1^\xea\xae\xfc}\xb1\x05\xb6\'|\xa2\x8d\xdf\xaa!\xbd1\x0e\xfcz-\x84\xba\xdca\xe0|\xe7#\x98\xa8n\x16X\xe3:\xadS{\t\x89\xd5v\x90\x90\x14\x0b\xca\xe2m\x88\xcb\xf5\xe3\xaem\xcf`I\xe0"\xda\x9c?\x14\xf5\xbcN\xd1\xfc-]\xb1~z\nL-(\x87Z\x8bH\xb0W\x8c\xa3\xa8Nm\xbd\xfcl\nY\xec\x08\xa1\xbfmM\xf4\xb3\xb5\x1b\xae_>\nw?\xd9\x0e\x96\xd6\n\x98\xb1\xc4\x9e\xa2\x95\x8e\xd0\x9b!\x9d\xf1\xac\xec\xce\xd1\x86\xd3\xe0\xe0-\x19\xecq\xed\x0e%\xf5;\tY\xa1\xed0\xfcp\x055K[\xc1*\xc7\x00_\xce\xe8\x8f-\x97&2y\xf5\xe1\x96\xe7\n\xf0\xf9)\xf0\xe8\x15\x9b\xe8\xd6\xaa^(\x0c\xed \x93\xd3\x17\x9d6\x0c\xd2au\xe3\x91\xac\xaab\xc9J\xc7\xb411q\x13\xcc\xf80\x8e\xfd\xfa\xfd\x03\xb7\xcf[q\xed\xec\x17\xb0\xda0\x82\x9ey\xcd\x83{\xf3\x8f\x92\xe9veXV\x16N\x16\xc1U\xa24\xe5\xa3S\xec\x94\xdd\x90;m3h\xfa\x85\x89\x7f\x1f\x1e\xa5\x01\x89\x8d\xb0t\xa8\x06f_\xee\xc9\x0f6\xa9\xe3\xc6\xc8S\xf0\xa4\xb7.\xbf\x91\xe6Qba89t\xfa\xd0\x9aM\x17\x04\xefU\xfd\xf1\xea))J\\O;\xd6\xdc\xb3\xa6+\x91=\xb0\xa2$\x14\xf2\x9f\\\x80\x91 \x87\x8d\xf3\x8e\x0b\xe1\xbd\xdb\xc4\xbc\xfe\xa6Xz\xbb\x98\xc6l\xb5F\xa9\x7f\x92S\x98\xc6\x0f*\xdb\xf8\x13\xaa~\x0f\xa0\x1d\xc7\xbc\xb0J\xb57KV]vJ\xdb?\x89\xb9-\x1a\xca\xac<\xd9\xa1\xcb~\xe1\xad\x82\x80\x92\xcf\x19b\xf6\xa3\xc3\x94\xec0S\x88Q\xeb\xc1\xee\xb9\xdba\xff\'Kl\xb9\xff\xdc\xe9O\x823\x8f\xe9\xdb\x07\xbf\x1c\xef\xc6\'{n\x05\xdd\xefc\xf9\xeb\xe13T\x15\xab\x0e&2\xd1N\x86\xfb\x92@z\xb5\x8a\x96}\xae\x85\xc9\xc6C9c\xf5qq\xc1\xb2^\xbc\xac\xcb1h\x9c>\x99|4\xbb\x92\xc9\xf6\xbe\xc2\xf8Ezx2\xa8/\'\xdf\xca\x17\x14\x96\xd9\xa0\xfa\x98\xcd0\xb1\x8f.\x96\xa6\xf6\xa4\x8dV\x0c73g\x81\x95\xe3y:1\xee\x17\xd5\xff\xf8\x08]\xebr!\xe2\xea0L\xdb\xe0\xcb]2\xcc\xb0\xe2m \\I\x9e\xcc\xee7\xcfAm\xa6\x19\xbf\x95\xfc\xa4 \xb4\x13\xf5_\xde \xd5~=(p\x86={\x9e\x1aA[\xe4\rQw\x8e:\xce\x9f\xd1\x1b\xc5\x99.\xd0\xa1\x90.J\xaf\xad\xac6Y\x94\'\xae\xd9\xd1\x17w\x8f\x07VYu\x82\xc6?2\xc7\x86\xe1ga\xdb\xfeAX\xd9\xdb\x9c\xed\xc2^\xc1\xc4jS\xae\x1f\xfc\x05RU&\xb3\x9a\xaf\x11*\x07_\x85\x96\xc43b\xb7r\x17vn\x08\'\xeej\xc6\x0ez\t\xb4?\xc4\x17m\x963\xf9\xda\x84\xd3\xea\x94\x16\xb8<{,L\xdd\xf5\x89\x84\x86\xe1`\xd2\xff\xb4\x90}([\xb8\x10\xf9B8\xb9S\x95\x13\xbe\xa4\xc0\xb5\xeeJ\xa8\xb9\xd4\x85Z\xee\x1c\xac\xb6\x1b\xf9\t\xf6\x8b#\x11"\x8c\xb8\x8b\xae,k\xbe\xf2\xe5\xf9RK\xf4\xbe\xd3\x0b\x1d\xea\xd2)\xa1\xe0.\xf8\x0e\xad\x02\xfd\xd1v\x9dqy\xb0\xde\xe7\x17pk\xcc\x08\xfc;\x0c\xf1\x90\xfek\xd2H\x1c\x84[\xf4\x95\xb0\xf0\xc8MR\\\xd6\x95\x83\xf6v\x01\xd5%\xc6$3\xfd+\xd54\xe6T\xab5(\xe0\x13\x87*p?\xd8\x0c\xd3\xef\x0f\x87\xc3\xcf\x7f\xd2\xefTK>\xbf\xf3)Y$(\xd3m\xfaM\x8d\xbdg\xd3o\x8d\xae\x18{\xe9"\xccl\xee\xc1J\xffH\xf0\x8a\x8c\x17\xf7\xb9t\r&=T\xe3\xfe\xf5CP::\xc4i\xdb\x18\x0b\x0c\xdf\xb4\xdai\x99M9\xf0\xc2\x0c\x983\x95@\xfe0\xe2\xcd\xb3\xe3 \xf1^\x05y\xf9\xbc\x83\x9fh\x85m\xcdC\xf9\x8f\xe9{\xd8yG\t\xc3\x12\xc7Ai\xfav\xd1\xcf1\x10\x9a\xbbW\x81\xf2\xb7\x1aJ\x88\xcc\x83\xeb]"@\x98\x95A\x82\x9d7\xbf\xee}\x93~\xcd\x1dL\x8f\x9e\xba\xb3G\xae\x16\x9e\x1e\xd7\x03\x15\x8d<\xc9\'r\x93X-\x8e\xc0_\xaa\xce\xb4\xbed\x0c\xae\xb4Q\xe3DUM\xd8wL\t\xe3S\xc7\xa2\xea\x93)$\xb9s\xb4\xda\xee\xb33\x9f\xba3\x1eO\xe9\xe8ax\xc7oa\xf0\xf8\xe3\x94U\xf5\x1e\x0cW\xbf!\x87Uk\x88\x1e\xbb\xf00S\xa7N\x9f\xd9\x95\xd5\xdd/\xc2\x86|}\x0cH\xd3\x82\xc7\xec\x84\x0f\xc6\xe8\xe1\xc4\x15\xf7i\xe3\x89N\xadt\xb9#\xcc\x91\x96Cv\xcc{\'\xb7\xb1Vd\xd5VA\xbd7\xb4P\x90\xfaJZ2\xd2\x1dlV\xee\x14=\x9fv\xd6\xcd\x8b\x0b\x8e\x9aA\xbdy\xd4\\{L{\xe3C[\xa3\x0c\xd0\xe0\x94\x17\x9b\xdc\x9a!4[m!\xf1z\x08}\xdd\x9a\x02\x13m\x06\xe3\x9f\x07\xdb!3\xac/\xcc\xeej\x83J*vx\xfdm\x1e\x0c\xfe\xf8\x89\xc6\x84\xaa`\xf6\x11\tL\xf5yF\xd2\xe5k\xaa\x1f\xfe\xb9B\x12O\xb5\xe1c\xa2\x0cqu\xab\x05\xd7\xebY\xf1\x92\xd3\x8et\xed\xe4q\xb2\x99\x1a!\xba\x19"?\xb8\xae\x88=M\xeeAh\xc56a\xe1\xd7!X\xd3\xf6\xa4\xda[f\x18\xbb\xffx\t\x0b\xfb+\xf3\xfd</8\xe5o\x84\xc9\x0b\x0e\x8bQ%\x13\xf1^\x9566u7\xe0\xdfK{\xe1\xcf4\x7f>\xf0!\x9evY\x94S\xc6\xdd\x12\xb1gd\x05\x99\x17\x1e\x80\x86F5v(\xd1\x80y\xf7M\xa0\xad\xe89\xfcz}\x8b,rj\x85z\x8bv\xaa\xea\x93AY\xa3OA\xed\xcdb:[\xf6\x81\x8cR\xde\xd0\x93G#\xd8_\xc3\x89\xdfm\xcf\xa4\xb7\xean\xf8n]w\xc6\x89\xa38\xf4\x9a3;7VB\xc0{\x1f29\xf2FlH\xbf\n\xd2\xca\x86jw%)\x9b\xff8\x08\xb5\xdb\x8f\xc3\xb4\x86KT\xa0{T\xb8\xb6\xd2\x9b?_r&\xd8\xaf\x87\xa3\xe4\x87\xf2F\xb768\xbd\xce\x00\x93\xf3\xbaq\xc4\x92q\x98\xdb\xa5\x03|b\x87\x88S\x15[ \xf1\\\x7f\xaa\xb9\x17M\x0e3[\x04\x83O\x07\xc4\xcf\x8a\x87ii\x0fO4k\xb1\xc2\xa2\xbf\xc1p\xfa\xa8\r/\xe9\x13\n\x8d1^\xc2\xdc\x8fjx\xd6\xc6\x0c\xc7\xfd\xee\x83\x83\x968aG\x9fG\x94\xfe\xfb\'\x1cz=\x06\x8b\xfb\x9a\xa2\xd4\xb6\x07\xe9\xb5o\x87\xc1\x0f\xfb\xe1\xe1\x0cY\x1e:w"\xe7-\xf1\xc0\x8f^\xc0\x162+\xe9\xa1\xc4\x85\x1d7\x9d \xd1\xc6\x9f\xff\x1ct\xe6\xef\'\xc6r\xb2\xecM\xc1\xf8\xa6:\xdb\xd7\x0e\xa5\xe4\xcf\xd5\x94\xd6f\r\xb2\x8d\x7f\xe1r\xa8\x05\xdd\xf6/\xa6\x1fskhP\xf9dN8+\xe16g[\x96;e\xc4\x13\x978\xe3=\xe5I<\x1bG!\xa4+\xe0\x92#>\xd4o\xb5\x1f\xaeK}\x02\xfe\x13[\xc5&o/~~W\x0b\x8d\xbf\xde\xa7\x8e\xf1\xa9\xe2\xfd\xa7C\xa1l\\\x07e\xffv\x14+T\x86\xf3\xb3\xcfcy\xd6O\x07\x96O\xeb\x89\xf9W\xbe\xc1\xcb\x85\x93\xf1\xeb\xe6S\x10R<\x8e\xca\xf6\xca\xf1\xc8\x18{\xfe\xa0\xd4\xd9\xb3\x8f}\x01\x8b\x1d\xf7\x84\xf5\xc75qa\x8f\xbe\x98U\xf3\x0er_?\x86w\x87w\x11\xe4\xc8\xf0eu3V^\xf3\n\xdc\xae[S\xd8\xdf\xbe\xe0O\x1b\xa95\xe7#Id\x8bD\x7f\xf9]d0t\x08\xad\xa0\xfd\xb0n\\\x12\xe5\xf6\xda\x06\xd9\xe1\xf3\x05\xc9&\x89c\x90N0\xdd\x19f\xc9#\x1c\x8d\xf1\xc62s\xbc\x17\xd2\x05\x13O{Ax\xbd\x96\xd0\x984\x92\xec#\xedhuU\n\xb8\x05\xc9\xf1\xfb\xb1\xa3q\xce?\xef\xe9@S\x1e\x19]\xa8 i\xe69\xd1\xf2\xa52\xc6=\x1eE\xc3f\x19\xe1y\xcf\x91\x9c\xdb=\x89\xdc\xbdU8A\xff1t\x0c~$*,\xb6A\xb9\xe3\xb6p\xacV\x82&\x87]\xc4gf\xae\x90\xa1\xffB\\\x9cg\x8f\xe7\xae\xf5\xe1\x82k\x15\xd4xE\x1f\x16T_\x85\xfa[\x87i\xaa\xeb\x17h\tJ\'\x9b\x83y\xc2\xc1\xc5\xe68z\xa8\x1e4I\xe5\xf9W]\x08\xc4xO\xe6\xe8\xee\x1b!\xfc\xddtq\xd49}\xfc\xf4\xd5\x84M\x0e\xcask/\x05Z\xf4d;\xdd\xfd<\x1e\xbd\x95{\xe0\xf5\x92d\xda4\xff\x14\xf5\xe4k\x14\xbf\xff\x1a-\x90/\x04\x93\xcfi\xf4\x06\xa6A\xe3\xac\xcdbN\x84\x16g\x7f)\xac\xfe|d\x1e\xf4\x9d\'\xc1\x8e\x8c\n\xc1\xe4\xd9v\xc1,\xb6\x99\xd6\xac2\xc5O\xf3]92\xb3\x9e2\xf7x\xc3\xbb\x80b\xf21?%\xaa>^\x01\x010R\x94Fm\x10TW\x19\xf3\xde\x81\x96\xf8+\xde\x97\x8b\xe4<\xe8\xfa\xfb\x16(\rT\x10\r6\r\xa7\xe8\x8a\x10\x983\xaa\x18\x9c\xaex\xf2\xb3\x19\xca\xdc\xcbr\x14\xdf8\xab\x80\x01\x82%\x87[jC\x85\xe5\x08j0\xc9\x80\x8e\x19=yz\xb8+\xcdQ\x1b\xc2{\x9a{a\xdd\xa3\xee\xe4\xa8v\x1d$\xc6\xd9N\xbb\x14\\\xf1\xa2\xa1\x06.\xd9\xb2\n\nGo\x83%g\x07\xf3\xe2N\xed\xccZ\x97\x0f\x9b\x1e\xba\xe1\xa5\xa7\nX[\xfb\x8eVO\xb8\x0f\x86r\x8f \xfdn\x13\xac\xbd\xad\xc7\x8bc|\xf0\xa1u\x03\x1c\xfa\xe8\x89\xe8\xbb\x97\x16\xbfl\xa7\x8d\xca\x17:}@7v;\xdb\xd9\x07\x94z`G\xc4\x15\xe1h\xd6{\xfaX{\x1c\x9c\xb3\x06q`X\x06\xd9h\xec\x12$\xc6k\x1dG\xff3\x1a*\xd4\x88\xa6\xf5\xf4\xe2\x03^\xedtK\xdb\x17\xb3\xef-\x16\xdb\xf6\\\x04[\xf9^\xa8\xfe]\x8b\xdf\ry\x06\xdb\xce\x8c\xc2\xcaH\r~\x14\xe1\x89\xc2CU\xca\xd7\xdd\rI5\x07@\xee\x19\x92\xfb\x93n\x1c.\x8c\xa5G3\xeb\xa8`\xbd/\x85<V\xe1==\xd51\xb9V\x8d\xc3\x8cwP\xe4X\x0f\x1eZZ\x0e\x9e\xcf\x01\xf6j\xb8\xf0\x91}\x88\xa3\xa7\x8d\x81\xdd\xd7\x0b\xa9(\xb47\xd4\\[Ou\x97\xc6\xd2\x06\x8b\x1b\xf4\xc1G\x1f\xf5\x92\'\xb2\x85y\x00E\xfc\xd5\xe0\xfc\x0f\x7f\xa1rF+\x0c\x88\xdd\x0bl)\xc3\xeeB\x0c\xcd\xc9m\xa0\x94\xc0\xc1\xac\xfe\xd4\x88\xb7\x85kq\xa8\\\x84P\xf7`\x04/8\xed\xcc?&\x1e\x85\xbba*\xf8yS\x10\xa8m\x1b\x8c\xddJ\xba\xe1\xcf+\xe68\xf5p\x0c|\xcfxI#\x12\xed\xb1\xec\xcf+\xd8\xb7n\x0cN\x9f\x10O\xafF\rg\x1d?g>\x17g\xceCn+\xe1-G-l\xf4\xf6\x17\xe6\xa6\xde\xa6?\xebt\xb88G\x13\xfd\x1f\x96P\x83v\rLz^\x06\xfb\xf6\x9bsQ\xf0I\x1a\xdd?\x81ny\xab\xe0\x8bD]\xb6\xabO\xa3\xe4Gf\xa2\x9f\xef(\xc8*\xb7\xe3QU~\xe8;\xa8\x16Z\xb2{\x89&Q\x9bI>\xd9\x0cW\xca\x98b\xd3\x83vJ\xb1\x7fE\xd2\x196\xc2\xcd\xa3\xa1\xf0.<\x07\xa6\xfd\xf6D\xef\x8a1<i\xf1=\xc8\xee\xadN\x05W}\xb9\xde\xe7=d\x9b^\x15f\xc6_\x82\x87\xbb#I9\xfa\x05h\xdc\xea\x8f\xc9\xb2\xd7\xc5^\xddmp\xe2\x80a\xe8\x1be\xc3jWG\xa2\xc2\xe6\xe1\xd8\xd1\x844\xbf\xbe/\xeb\x95?\xa4\xc2\x7f\xf6\x82Ie7\xe1\xad\xa8\x8a1]\xcd\xb0|\xad*z6h\xd1\x9c\xc3/\xc9R\xad\x8ev\xb5\xc8`\xb2\xf10q\xbd\xab2\x86\x07[\xf2\xd9\x1c\x07~\xbc\xf8-U\x1e\x8c\x87\xd4qo;s\xff\x10H\xf3I\xac9^$Hj\xba8\xed\x9b\xe9\xcb\x05\xder\\x5\x078T\x93\x13\xbe\x9f"\xc9\x80\xe7\'V\xeb\x1c\x84\xb8%\xa5\xf4Wq\x10\xce\x1c\xd6\x83K\x1c"a\xea\xf16\x18\xfa5\x95\x8c^\rE\xcbi#\xb9p\xcc{\xd8jw\xaa\x93}\x1f@\xabF\x18\x15?\xf4D\xbbmm`?\r9\xee\xdeU\xaa\x99\xda*\xd4m0\xa0\xcc~R\xa8=?\x88\x83\xd5C\x825\xe6\x04\xca\x04\xfe\xe74\x8af\xa0\xdc\xe6\xe0n\xbe\xc1Z\xff\x1eE\xd1\x8e\t\xd6\x0eT\x89\x1f\xe5)\xf9?\x1c\xc1:!\xc1\xba\xff]O\xef\xdf\xf5\xf4}\x83\r\xfe\xb3\x9e\x87Rlp\xf7\xce\x15\x9d\x8bd%\x92\xa0\xf5\xfb\xe0\xbe\xb8\x1a\xafo\x8b\xc3\xfb\xf5\xe5x\xf1\xf1A\xb6\x08l\xe6\xa6&}\x94t\t\xaa\x0e\x9a\xfc\x16\xf2F\xcd\xe1\xe1\xb7\xaf\xb0N\x98\x13;\xa8\xcd\xa6\xa6\x0c+|\xb5\x878ym\x1d4\x9e\x91\x81\xf4\xa3\xb7\xf1\x97\xbf<N\xaa?\x82\xc6.1\xec\xf5|5O\xfd\x90\x8c\x12\x8dd\xb2\xb1j\xc4\xddr\x838N7\x13n/\x0bG?\'\xc0\x92\xfb\xd1\xf8\xd2-\x81\xe3F\x9b\xe0\x92k\x07\xb9\xc7\xa2\x1d\x1c\xfan0&\xf5n\xc5\x96\xbbjh\xb3\xf8\x8dx!l6\x9d\xde\xb1\x94\x03F\xa6a\x81\xfa\x0787a\'w\xfc9\xc4}Z\xaf\xa0\x8d\xbd\rKd2\x18}\xae`\xb8\xccf\xc2\xd6\n\x96\xc6\xfa\x8b\'\xbb\x1db\xb7\xa3\x11\x1cd\x98*\xce\x1bR\xc9\xbb\x157q\xc72o\x92K\x8dB\x7fw\'\xb6\xebq\x0e\x9f\xbc\xb0\xe5p\xb3]\xe2\xb7\x9a\x87\x1c-[\xc6\xd9\xde\xf7af\xef\xfe\xec\xfb\xbc\t\x03\xbbI\xd1\xefB\x0em\xb4\x8f\xe4\xb8.\x8f\xc1wo\rf\xacT\x87\xd5\xe7\xefa\x9f\x8aZT\x8d\x9d\x87-\xbe\xdb\xd1"\xf4\x00\xdf1\xbb\xcf\x05\x1d!\xb8q\xb6?Kj\xd7W\xfb\xdfN\x84\'\xa5\x17\xf1\xe2\xe2\x06\xce.(\x82\x9a\xef\xc5B@\xe4\x05\x88\x0e\ra\xff\xef\xcfp\xf5[]\x1e1\xb0\x91\xcf]i\xe4\x93\x96\xcf\xf0V\xe1LNZ\x90\x81\x8a\x8b\xf4``\xd4v\x1cm\xbf\x13=\xef\x1e\xc2\xd0\xf1{\xc1w\xd2a6Y9\x03\xb3\xfdO\x91\xe6\xd8X\xf1\xfe\xd1\xed\xac\xdei\xda\x85\xe1;0\xb1v0z\xad\x89\xe1\x8c\xf5/\xe1d\xbe\x88>)\xc7\xd9d\x8a:\xd6\x8f-c\x83\x1f2\x9d\xe0\xbc\x8b\xbe\xe9Er\xd3\x1fy<\x19\xd8\x86\xe9\xba\x07Y\xdas\x1f\xb7\x87\xf4\x83\x16\xeb"~a\xa0\xc6\x92\xa2a,\xb9|\x11\xa4g/\xa0\xc6\xdc\x9b\x9c\xd4\xab\x01\x17\x9c~\x8a+^\xe7\xb3dS\x05.\xf6\xda\xc2\xd2;A8\xbd\xf9%\xfadMC\x9c\xf2\x82}\x07\x1fE\nO\xe17\xe3\xfbb\xed\xdb\x00\xae\x11\xf6;]\xee\xd0\xc3_\xba7(\xa8\xff@^\x12f\x81\x07\x9a\xb7\xf2\xf3\xf2\xcd\xfcE>\x94\x7f\x94\xa7\xa0\xce\xb5hL\x18\xdc\xc27\xe7\r\xe3\xf6\x9e\x9310%\x81\xd3\xba\x8fa\x837z\xf4p\xe2\x1et\t\xac\xe5\xec\xabV\xd0QX\x86\xf3\xb4l\xb0\xb1\xc2\x12V\xdb_\xc0\'\xa1;P\xf9\xd6}t\xf7\xe8\xc9\x1f/\'\xe0\x8a\x96]\x1c4\xed4\x06\x1b\x86\x04\x1b\xfd7\xa9\x8d\xffM\xea\x1e\xbe\xc1=\xffMj\xa5\x98\xe0^\xff\x16\x91\xe9\xffWq\x98\x84\x04\xf7\xfe\xef<}\xfe\x9d\xa7\xafo\xb0\xe9\x7f\xe6\xf10\x8f\r6\xfb\xb78\xfc\xe5%\x92\xb8fu\x96\xd6\x85b"d\x92\xa8\x94Iu\xca\xa5\xecc\x1a\x05\x8a\xd7\x1fCk^\x04gD\x0f"\x9e\x1d\x8ev\x92\xf9\xbcbU\x16\x8e^U\x0f\x16\x81\x9a\x98\xfa<\r\xdd\xf6\xccc\x1f\xfb\xc5\x80\xfdK\xb0\xdet\x12\xab\xf6P\xe1y\x8f\x05\xbc49\x97\xafz\x94\xb3\xf4R\x1aY|^\x8a\xd2\xec\x13$\xf8|\x86\x8c\x11\xc1\xe8z\xa7\x14gTH8\x7f\xc9\x16,\xc8\xaf\x03\xb1l\x1a\x9a\x9f\xde\x86\xe3\xde$r\x8d\xd1\x14.\x15&\xe3\xbd\x97#\xf8\x82\x8e\x08\x95{\x931S\xfb(\x86\x85g\xb1g\x974\x90\xbe:&x\xb9\xefC\xb7\xa8=\x98<\xa0E\x9c\xb4!\x01%_R\xab\xa5qv\xe8zW\x195\xb3\xbbs\x8d\xe9|\n\xbc<\x8b\xc2\xf5\xce\x88:\xd6\x13\xb9\xac1\x9d\x03{\x8f\xe4\x8e\x82lr\x96\xed\x86\xe5\xf5\xb9\\{~\x07\xbe\x9al\xc05\xef\x9fUk\xb6\xd9C\xd7\xf2D\x96\xf3\xde\xc0\xe1wu\xc4h\xf9t,\xeaw\x10\xeb\xa7w\x90\xd4\xef\x86\xd3\x9c\xa9i\xa8>g\x1bK\'\xad\x01\xc9\x80\x04\xa8\x93\xac\xc5\xc6\xf0f\xc1\xad{7\xac_\xdaN\xb6K\xf7\xb1\xdf\xf1Rpxk\x87\x03\x16D\xe1\x80\x17#Y\xfa\xb9Y\xe8\xa9\x9c\x81\x92\xd8v\xa7\x88\xf7\xd1\xfc\xa3\x87\t[LX\x0fB\xee\x1a\xfe\x15\xed\x88\xaa\x8d\xeb\xd0\xffn\x13\x15l\x1c\xc1\xf5!\xfd8\xb3\xeb\x08lL\x1e\x04Z\xdf\x0e\xa0\xb1c\x0e\x8c\xde\x9a\xcb\x8d\x03\xb7\xa2\x7fB\x1e\xae\xef\xde\x9f\x93\xf26q\xfef\x05,\xfa\x95\x88\xeau\xc7\xd0%`\x06g\xac\x92\xa1\xd0\x13\x03\xa0\xa5\xa5?\xbf9q\x0cZ\xd3r(th(\x1f\x88\x9b\x8c\xfaS]\xd9 \xa3?Z\x04\xd9\xe2\xee\x9d\xdb\xb9k\xdcN\xec\xbaP\rSeK1\x8dR\xe1\x90\xba;\'\xe6~%\xc9\xd3\xad\xb8\xfaW2z\x8e\xc8a\xcd\xaa\xbb\xf0\xf9^:\xba\xf5\x19\x88\xfasr9.u\x1fW\xc5\xf4@\x93\x15\xb2$\r)\x85\x9a\xd1\xee\xb8\x1f\x1c\x18\xae\'\xb0\xc1\x99\xceg|\n1\xf3\xa8>n\xf3\x118b\xd2\x14\x0e5\xdb\xc1v\x93\xa3q\x89Sw\xbc\xb0|\x03\xfa\xb4\xaaAG\xd2I\x08\x9c\xb8\t\x93o\xfb\x93\xcf\xb4x\xbct6\x9f\xab\x9e\xee\xc1?1\xbd8\xee\xbc\'\x14}p@\xf3\xb6\x12\xcexu\x83\xc2no\xa3\xe8L3<\x90\xdf\x06KT\xd7\xa1\x14\x7f\x8b\x8d\xdfG\xd2\xfd\t\xce\xe8o\xdf\x1d\xe6\xd9&r\xc1\xe1\x100\xeaY\xc8A\xc5\x07Iz4\x9d\xb6\xdc8\xc0\x92\xa01\'\xedo\xa6p\x80\x9c\tI\xc7/E\x83\xe3\x0e,\xf90\x1d%I\xf9h|k;\xd2\x8e\n~1\xc2\x8f\x15\xd3bq\x9bV\x17\x1e\xac\xe5\xc5\x06\xd0\x9b\xa6\xa7l\xc7\xf0\xab\xaa\xe2P\xfbV\xdaP\xbd\x9dok\xcf\xe2\xe0\xca4V\xbd\xb7\x08\x13\x1f\xbf\x85\xd2I\xff\xe0\x8a\xc6\xae\x98Q>\x0c\xc5\xd0~X\xf8\'\x923\x16\xea\xe2\xf5\'\x19\xd8~\xf2\x87`\xbf\xe6\x04\x94\x96\xeaQ\xadM"\xde\x1f:\x18S+Jy\xc1\xcd=\xec [\x04\xbf\x9c\x7f\xc3\x93\xf7\x87\xb9ev(%\xcf7\xecl\x1c\xb9py\xc36\xc84\xd6AIC\x1a\x87_\x1e\x04M\xeez4#\xc8\x8e]\xb7\xef\xc6\xdbU\xb9\xd8v4\x1a\xdd\xbc\xac\xd1\xcf\xae\x1eB:\xa92h\xaf\x14\x9a^\x17R\x90\xfc9\xaa\xb3\xdf\xc3:\xceS\xb0\xe6\x91!~\xadK\xc6\xf0\xc41\xd4\x8ei\x82s\xed~\x8c\xbe\xaa\x8cir)\xd8\xfe\xb6B\xc8\x90\x16\xe0\xe6\x05\x96|sp\x11\x1a\xaf\xcbb\x8d\xa3R\x1ewM\x99K\xca\x8ep\xb6\x8b\x83\xb0\xc8{\x0bj\xaeo\x10\xa4\x8b\xaf8]V\x9a\xcf\xae_\xfa\xf0\xf3jM\xcej\xcacc\xdftl\xde\x97\x86\xe1\xef\x8f:\tOW`\xcb\xc9x(\x1a\x9b\x06\x92\xdcy\xdcnZ#.\xca\xdc\xce5\x8a\x9e\x94h\xa6\xc0\xc6\xd1\x95\xac\xa0\xe7\xc7\xcd_\x0fcV\xf0vNX)\xc1F\x84N!\xb5\x02s\xd3\xad\xa8\xf3r\x06\xc6\xcd\xcb$\x9f\xe3-Bj\\\x05\xb6J\ny\x85{\x19\xd7l8#4\xd6)`\xaey\x02\x9e\x1d\xa4\xc9\x85s+\xd1\xe6\xbd\x0e\x1cL\x1d\xc8\x1b7\x96\xe2\x82\xdb\x9f\xa9\xa0\xe3\x0b\xdc<\xfa\x1dL\xe6=\'\x8b\x95\xcd\x906.\x12\xde\x98\xe4\xe0\x07_->v\xad\x9ck\xbf\x8eg\xe3\xfe\x11\x98\xad\x1e\x06R\xfd%\xe2?\x03M\xb8\xa9\xbe\x94C\x9f\xae\x02\xcd\x80\xee\xec\xf0\xc0\x0e*\x17\x96\xa0\xb1\xbf\x1f\xda\x97\xa4qv\xac\x1f\x95\x1a~\x06\xc7\xe9\xe9\xe8x(\x1eo\xeeN\x85\xcb\xc7\x97b\x90\x9f/\x97\xeb$q\xf2\xe5$\\qm\x1f\xda|\t\x87|\xe1\x10J\xc2\xf7`v\x92\x05\xb7\'[\xa3\xc1\xe0q\x94\xad\x1cA\xeb\xc6{\xb0\x96\xcc1,\x93\xc9\xc6\xe4\x83OH\xd5l;\x0e\x0cz\x005\'\xc2\xe9db\x0c\x9f\x13\x0e\xf1\xad\xfcX^%hse\x9d\x0c\xbe\xbcY\xc1\xc1\xe6!\xc1\xfd\xfe+\xc0\x16\xff\np\x7f\xdf\xe0\x01\xff\n\xb0yL\xb0\xe5\xbfB\x9e\xf1\x7fIC\xff\x97#x`H\xb0\xd5\x7fc\xb7\xfe7\xf6A\xbe\xc16\xff\x89\xfd\xdfC\xc5\xb6\xff6\x0f7\x15\x89\xe4\xa3\x86\x1a\xf6Y\xe4\x82\xfdl\x1dy\xcf25\xd6o\xdf\x8eg\x86D\xe3\xf2\xf7{\xb0tz\xb8\xa0\x13\xde\x13m\xe26\xb2\xa4\xe6U\xd5\xa3\x93q<s\xefx\x14\xafg\x83\xfe\xe5\x18\xac\xcd\x9e\xc1\xe1\xe2X\x18V\xb7\x15\x83\xf3\xe2\xb1\xc8z\x07gJ\xd29u\xcaH\x9cVt\x00;\xce\xaf\x81\x8cw\x190@\xb3?fFo\x81\x7f>\xa5\xa0\xe8Q\x0f\xc9\xa6\xda|\xb8\xefT\xfe\xf2\xd5\x83+\x1cM\xf1\x99\xf5\x0e\xb8\x02y\xb8$(\x87\xca\x1eg\xf3f\xfb}\xfc\xb6%\x9e\x8d\x97\x17\xc3\xbd\x0b\xf1X\xdf\xfc\x92J7]\x17_\x8c\x98\xc9\xafz$\xa2\x81s<\x94\x1f2a\xd5\xde}q\xc2\xcfmX\\\x98\xceKw\'pU\x9a\x07\xbd\x1e7\x84\xfb\xa8\x01\x1e\x9c\xb5\r\xd7\xdd\x98\x80\xc7\xbe\xce\xe2\x9dj\x89\xf8Q\xafS\xf4\xed\xba\xc3\x83\xa7\xc3\xd8O\xf3&\x9cX\x14\x8b\xd2#\x03\xf0\xa1w?\x9c\xb7\xb17g\x84Z\x93\xd4n\x95\x90=\xea\xa0P+\x99\x8a\xdd7\xaaq}\xf3b\xae/\xb6E\xf1\xd1\\V\x8a\xedL\xaa_\xd9\xfc$c\x15\xcb<\xcc\xe6\xdc-\xab1t\xb8:\xb5\xcb&\t\x13s\xd6p\xc6\xc6\xab\x10\x99\x1f\xc6v[\x13\xf9\xbbU\x06^\x08\xcd\x81`\xfbx\xac\x9a\xa1\x08&9\x0f){\xd8C\xb1\xf4I5M\x86}\x18\xfeLW<\xbbi/\x9f\x8c\xe8\xca\x06\xd3{R\xf8\x9e\xe5pn\xffb\xbc\xf37\x17\x9f\xcc,\xa5A\x9aIX\xdb\xed\x00\xa5\n\x95dS\xfcLlk\xb4\xc7.\xfd\xc7s\xa6B\x1ehj\xaf\xeb\xfc\x0f\x12\xd9\xc6\xc6\x0b\xcc\x8fL\xe7A\xd3\xb3\xd0u\xe7d\xcc\xc8QE\xc9\xae\xee\xd537\xfd$\x9d\x15\xa6\\\xdb\xe4\x84F\x07b1yF\xb60\xf0E"1\x1c\xa6\xeb\xfd\xa2\xf8f~\x08\x87\x9de\xf8\xf3\xfa\x07\xcd\x92\xcf\xe7H\xdf\x03\xec\xa5\xba\x98\x0b\xae\xa8CX\xcdt\xce(>\n\xfb\x7fE\xf2\xb4\xe3\xb9\xf8!"\nWW\xcd\xc3\x0b\x03"\xb9w\xf2v\\\xdai\xa7\x95\xbe\xeeD\xc5\xcc]\xecS\xb4\x825\x03\x9f\x89\xde\x82<\x9b\xa8\xc4\x8b\xf6\xd7\x12h~b\x04\xbf+O\xe6\xd4\xb12\xa8\xb5t/\xdf\xfc$\x92\xf6\xa98,\xbd\xec\xcc{\xcfkp\xbf\xdb\xdb\xd8p\xa2\x0b^\x18\xb2\x89\xda6\xed\xe6\x80\xd3&p\xd3>\x9a*N7B\xee}{4\x1b\xee\xc2>\xd9k u\x80\x12\xffr\x88\x83\x01]s\xc8\n\xf7\xa0\xa9\xdbAj:\x95\x01\xc1{\x12Xn\xed7*\xd0\xfcI\x9f\x93;\xe9\xb1\xc3\x96\xe2\xb3\x81i\xa7\x1c\xbf5\xb6\xe2x=+\x9ei=\x15K\xdf\xfc\xa2\xa2\x8es\x14p,S\xd0\x1c\xbb\x182\xa6\x0c\x82\xda\xb3\xb6\xdcX8\x8f\xb3\xbfg\n\xb7\x83\xc6\xa3\xaeC&\xe7?5\xe0\xd6\xe2,\xf2MW\xe4\x86\xaf\xe38\xe8M?z\xbcu\x0bj\xbcpe\xe3\xf5y\xbc\xcf \x0f\xe7m\xcae\xfb\x13;\xf9VE\xa7\xb8R\x00\x86\x1b\xac\x85&\xb9\xc3 V^\x04\xcd\x03M\xe0sT\x0f\xd2\xa22\xc0\xbal7\xe9-\r`\x1f\r\x15(\x9f\xba\x19\xbb\x86H\xf9\xed\xe6\x9d\xd8\'~3O\xbfV\t5\xe3I\xac2\x8e\x01\x13\xb6\x13\xb3\xad\x86\xb2\xe1\xb5\xa9\x18v6\x1eBC\xbe@\xbf\x99Yxe\xc9^\xdc\xb0+\x19\xbfD\xca\xb3\xd9\xe3,<\xeb\x97\x8bA_\xcd\xf8\xd3"U.\xdd\xf7B\x90n\x18Gq\xef\xbbp\xe5\xc1\x0c\x9a\xd4{\x0b\x1bu\xdd\x89\xa1r{yb\x97d<V\x15\xcf\xfa\x0b\xf20\xf3b\x05\xf4>\xb4\x1b\xad\x9fO\xc5\xf7\xe3\xfap\xb7\xb2q\x9c\xb2\xc6\x9e\xcbK2\xf0\xdc\xd0M(w\xea\r}\xf2\xdb\x85^\x0b\xc6\xe3\xa7\x9e{\xb0qB%\x84\xfay\xd0\x81\xc1\x02k~\xb5\xc6\xf0\xf9\x9b\xc5\xe4[O\x84\x90&\x11\x0c\x7f\x94u\xd6C\x0e\xcd\x8cT\xe5\xf0o\rt\xaa*\x1c\xcb\xbaz\xa0I\xff\xdd<r\x8b,\x87\x0fm\x00\xc9\x0b\x15\xb1cko\xd4\x9c\xb7\x88\xe3\xaey\xd0\xc7\x9c`~\x7fM\x8e\x97\x81\x07J+t\xe8\xcd\xf4\xa3\x90\xfd\xb6\x11\xd6\xae\x9e\xcb\xdd+\xb6\xe3\xe7\r\xfb xC&+\\\xb4\xe3\x99\xdf\x0bI"\x89\xac\xf4j\xbdG&\x1a\xa5\xe2\xee}R\xac\xfa0\x90$\xf9\xf2\'3\x07\'\xf2\xbb O\xcc\x1eU\r^\xf9Ky\xfa\xb3\xb9|]o\x0f\x7f\xbe\x7f\x94\xd6\xce\x99\xc3\x93ZW\xe1\x8c\x13y\\\xf0 \x93\xadsza\xd2Z/,U\x8d\x11T\xbe\xeff\xf5W\xa7\xa9e\xe8Q\xf8\xfb:\x05w\xd6\xc5\xf3c{u\xae\xb3\xd9\xcb\x17o\xc6\xa1t\xe2?b\x8f\xf6x\xae\xb1\x9b LI\xdc\x8f\x15\x17_C\xdb\x90\x15\xbc\xfc[\x16z]\xec\xc5\x8b\xbc\xed9{\xab?\x19\x16\xcd\xc1\xf4\xa3\xbb\xf8\x87\xed-2\xe9\xd9"\x144\x9a\xc2\xfeK}\xb8@!\x94Lx\xa6`\xd2_\x86\xbd\x0eOe\x13\xb9PN\xdf8\x12\xef\xdb&\xb3\xd2-7\x9ey$\x9e9o\x14\xb6\xee\xffD\xc5+zrV\xff\\\xde=\xae\x99.\x1c\x1f\x0b\xcaw\x1dp\xe6&{\x94>\xef+V\xe5\xce\x81\xfd\xd3v\xa3\xab~\x19\x05\xdc)\x14F.\x8b\xc1!\xdd\xf28(\xe3\x1a\xd5\xc8/\xe2\xb9\x0f\xd3Q\x87\xe5P{\xc0.\xec\xd0\x9d\x05\xcd\xfb\xf3\t\x8fM\xc2\'\x13\'b\xd8\x9aW\x90\x1a\xe4\xc0\xc9\x175\xd1\xb9\xaa\x85\xe2+vs\xa8y&\x07\xf4\xb5\x11j>N\xe3\x8d\xae/\xe8c\xa9>\xd6~q@\xcd\x9eG\xc9!\xa3\x0b\x8f7\xc9C\xd1k\x06\x83\xbe\x1e\x17\x9e\xfbI\x19?\x16\xf2\xe8\'\xaa\xf8mG\x04\xfe\xf1\n\xe6\xf0\xd5\x96\xc2\x86\xe0N\x13Q\xd9\r\x8e_s\xe1\x8a\xc99\\cUW\x1d\xfeR\x14?$\xb9\xb0\xeb\x8d:r\xfb\xb9\x8c$\xd3n\x08/m\xbd\xb9\xab\x8d\x11[\xab\xcc\xc5\x83\x0bs1yR%=p\xd9\x8b\xedF\n\xe8\xf43\x02\xbbi\xaf\xe7\xa0\xe4\xe7\xd4o\x8d\x0bG\xff\x8d\xa6;=\xbbs\xba\xed&>\x14\x9b\x84OtV\xa0$nm\xf5\x86N\x18\x97\xc0\x13\xa7\x0b\x19\xf7\xa8H\x98\xca\x158\x99\x0b\xf7X\xe3\x85\xb9\xc3(c\xdff\x9a~h9KO\xbf\x14\xcc\'M\xc7\x19\x89\xda<\xf6h\x1a7\xbd\xb2\xe0\xdd!\x9b\xd9\xbc\xeb@l\xf4\xbfK\xeb\x1c\'\xa0\xe4\x07\t\xe2\x88x\xfa\x18%\xc5\xda2\x03\x0e[\xfe\x1b^\xf6\xcbE\xcd\x8b\xb1<:\xe224\xc6\x1c\x12\xe5\xa6\xacf\xcd/\x96\xa0\xb5\xad\x17>\xb91\x125+\x92\x85\x03\x1b\xf6\xf1\xbe\xd0-\xbc\xf1I\x1c\xb7\xdc\x8ce\xd9Y6\x18\xda\xd8\rC?ks\xba\xce|\x9eU\xa9\xc0\xd9\x95\x8f\xc5\xb8\x8f\x7fA\xc55\x07\xdfN\x91r\xc0\xcbbQ\xba\xa7\t\xe6\xeeT\xe6e\x95O\xe9\xfa\x9c\xf9\xe8\xf9\x8f\r.\xdc\xbc\x9a\x13\x95+\xe0\xc1\x84\x9d\x18\x9b\xe1\xc1\x9a!\xcdb\xfb\xf4\x1bbsk\x12\xf7R\x8bC\xeb\xa6O\xd4\xbe\xaa@\xc8\xfd\x18\xcb-NR\xce\xee\xfc\r\xdb\xb3>\x80g\x9fO\xf0\xf7D:^\xf8`\x07\x99\xd7Z\xa9\xf1\xe6\x05a\xf4\xa9Z\x18<*\x1a\x93\x9f\x1b\xc1\xbbA\xbe\xdc\xed~\x047]\xd9F\xd3\x9e\xed\xc45w"\xb0b\xd1J.<\x18\x86\xd2{\x13@\xc7\xbd?\xbe\x08\n\xe3p\xe5\xd1BFT!\x1dk\xdb\xcf\xee\xfa\xe39\xe4\xdeS\xb2{\xb0\x04M\xe0/L\xd2\x9e\x8eoV\xbf\x86\xec\x81\x9b\x84\xd1\xda#\xf9H\xa5\x05\xef\xcb\x04V\xcdM\xa4\xd1\xd77s\xa0K+Uv\xd9\x84\xb2]\xfe\x92V\x807\xe6\xee\xb6\xe6\xcc\x17\xdbx\xd4\xddD\x1c\xe2\x1a\x89\x17*\x14ha\x98\x12\x07\xd4n \xadO\xb3Y\xbbo*6FE\x881\x01\xa39\x93s\xd8\xa7\xf8\xa3 V\xbb\xb0\x8dd$D~\n\xc5\xdcoy\xbcR\xf5\x00*\x1f\xf9A\xa9;B\xd0\xc7U\x87\x1d\x06\xf4\x84\x13\xfav|b\xad\x02\xeb\xa4*bb|)-\xdf\x90\x88\x81N\x97(\xbb@\x81k\xce\xbfp\n\x9bg\x84\xa5\xc7\xb5\xa0yw&\xaf\x1f\xaa\xc0\x17N\x10}==\x10\xa7\xdf\xb3\xe6I?\x96\xa2E\xb1*d\xb7\xd9\x8a\x1b\xce\xf6\xe4\x8d;"9`\\\x81\xa8\x95\xe0\x89\x92=\x8b\xa0k\xbe\x80c&\xef\xc3\xe6\xf1\xab\xb1\xb5Z\x91\x0c~\xef\x04\xd9\xf2 \xf4)\x9c\xcfM\x1fU\xc9\xe4\x1f3\xf8R\xb6\x85\x83R^\xd0\xe5\xb9}Yr\xc0O\x08x\xb1G4)^\t\xfbK\xe6\xf0\xb3o\xd1\xe4\xfa}6\x86\xbe]\x0f\xb1\xcd\xee|g\xe0.N\\q\x992\xe7\xf4\xc0\x92#\xf3Q\xf2gu\xb5\xdf\xde\x16\xf8\xbbn\x1b\xd7\xcc\xdd(:_\xed\x8d\xe7[u\xd8\xcf\xff1\xf8\x98\x7f\x12\xcf\xb4&b\x1fw\x0b\x9c\xe2\xb2\x8f\xaf\x87\xeeb\xdd\x07\xd9L\xed\x19\xd8\xe2\xa1C\x05\xb7\xdc\xa8\xe6\xc1\xadj2\xb4\xe5\xe4\tQ\xe2w\xeb\x0c\xb4\x96\\\xa7\xc4\xaf\xd7\xa1\xe0\xe6|z\xe9\xb3\x1f\x9f\'\x08l,\xe4\xc0\xee\xe4\xd3T\xb4\xfe:\xf5Yu\x8f\xf4\x07Ea\x83\xe9^:\xe7\xe9\x82\xcfEW\x0eX\xfc\x87\x06vM\x06\xe3\x07I\xf0\xe1Qw\x1e\x11\x9b\x8bV=\xb20)l\x01\x1a)ga\x8d\xcf^\xb1\xe7\xcb\xdd\xac\xda\x12MIj\x91,\xc4\xe9p\xe3\x91+\x14\x97x\x08\xfa\xec\x11P\xb2sX\xb5\x9d_-YuB{\xf84\xc5\xea;)\xbd\x19K\x14p\xca%m\x0ep?,\x1a(\x1d`\xd5f=\x96U\xcdd\x83\xad=Q\xd1\xa9\x1d\xf2\xbag\xa3dM_\xd1xW"OZ|\x8eJ/\xc9\xd1\xb25C\xb1\xef\xd0D\xd42\x99\x8e\x8fKSQs\xdakx\xfbu\x17F\x9f\t\xa7X57\xce\\\x9e\x05>2\x11P\x10T\x0c\x9aZ\x0e\xd4\xfc\xbb;\x0b[\xcc\xf1\xdb\x98\x1e|\xe0U-\xd4\x04\xc5p\xdd\xf2$\xd2\x1a\xb9\x9b"2\x16\xf3\xcf6G\xbe\x12\xd5\x9d\xff\xe9\xe7\xc93\xd7\xe8r\xd4\x80\xdd\xe8\x13P+D\x9e\x08\xe7\x87\xde\x96x\xefV\x16\xeaj:r\xa0\xf6f\xd8C\xfb1h\xda{\xf1\xb6\x9cH\xcb\x96Y`\xda\xe8\xe5\\\x13{K(\xd5R\xc6\xb8\xd2/p\xb7\x93)\xdfso\x0e\xcd\x9d\xc3\x16\x11_\xa8\xd0M\x8a\x16\xa5\x89\xec0\xbd+\xcd<\xb9\x96\xc3&\xae\xe26\x0fG\x1c\xf8#\x85b\xb2\xd2\xb0;t\x1a\x90\xf2\x9dl\xd3\xa9\'\xffx\x85\xa3a\xcc\x18\xf4mq\xc1\xd7\xf3w`k\x826\x1f\xd3LG\xad\xf4$^\xda\xc7\x8e\x87\xbf\xcc\xc5\x86\xf9\xb6\xfc\xa7S\x95u\x82\x82\xb8\xfb\xc9t\x9c\x94\xa4\x8f\xd7\xd5\xaf\xd3\xd1\xe5\x19\xd8\xcd;\x8f\x83\xedB\x82\x07\xff\x17\xb8\xed\xff\x05\xee!\xbe\xc1C\xffw\xebt\xd8\xff\xfd\xd6\xa9CH\xf0\xf09\xf3\xad\xff\x1f.\x8e,F'


class DenseNN:

    def __init__(self, observation_shape: List[int], n_actions: int, 
                 output_activation: Union[None, str] = None,
                 unit_scale: int = 1, learning_rate: float = 0.0001, 
                 opt: str = 'Adam') -> None:
        """
        :param observation_shape: Tuple specifying input shape.
        :param n_actions: Int specifying number of outputs
        :param output_activation: Activation function for output. Eg. 
                                  None for value estimation (off-policy methods).
        :param unit_scale: Multiplier for all units in FC layers in network 
                           (not used here at the moment).
        :param opt: Keras optimiser to use. Should be string. 
                    This is to avoid storing TF/Keras objects here.
        :param learning_rate: Learning rate for optimiser.

        """
        self.observation_shape = observation_shape
        self.n_actions = n_actions
        self.unit_scale = unit_scale
        self.output_activation = output_activation
        self.learning_rate = learning_rate
        self.opt = opt

    def _model_architecture(self) -> Tuple[keras.layers.Layer, keras.layers.Layer]:
        
        frame_input = keras.layers.Input(name='input', shape=self.observation_shape)
        fc1 = keras.layers.Dense(int(self.observation_shape[0] / 1.5 * self.unit_scale), 
                                 name='fc1', activation='relu')(frame_input)
        fc2 = keras.layers.Dense(int(self.observation_shape[0] / 3 * self.unit_scale), 
                                 name='fc2', activation='relu')(fc1)
        fc3 = keras.layers.Dense(self.n_actions * 2, 
                                 name='fc3', activation='relu')(fc2)
        action_output = keras.layers.Dense(units=self.n_actions, name='output',
                                           activation=self.output_activation)(fc3)

        return frame_input, action_output

    def compile(self, model_name: str = 'model', 
                loss: Union[str, Callable] = 'mse') -> keras.Model:
        """
        Compile a copy of the model using the provided loss.

        :param model_name: Name of model
        :param loss: Model loss. Default 'mse'. Can be custom callable.
        """
        # Get optimiser
        if self.opt.lower() == 'adam':
            opt = keras.optimizers.Adam
        elif self.opt.lower() == 'rmsprop':
            opt = keras.optimizers.RMSprop
        else:
            raise ValueError(f"Invalid optimiser {self.opt}")

        state_input, action_output = self._model_architecture()
        model = keras.Model(inputs=[state_input], outputs=[action_output], 
                            name=model_name)
        model.compile(optimizer=opt(learning_rate=self.learning_rate), 
                      loss=loss)

        return model

    def plot(self, model_name: str = 'model') -> None:
        keras.utils.plot_model(self.compile(model_name), 
                               to_file=f"{model_name}.png", show_shapes=True)
        plt.show()

tf_mod = DenseNN(observation_shape=(115,), 
                 # This needs to match the number from training
                 unit_scale=0.25,
                 n_actions=19).compile()
tf_mod.set_weights(pickle.loads(zlib.decompress(WEIGHTS_BYTES)))

def agent(obs):

    # Use the existing model and obs buffer on each call to agent
    global tf_mod

    # Convert these to the same output as the Simple115StateWrapper we used in training
    obs = Simple115StateWrapper.convert_observation(obs['players_raw'], fixed_positions=False)

    # Predict actions from keras model
    actions = tf_mod.predict(obs)
    action = np.argmax(actions)

    return [action]

# Test submission.py

The written submission file can be tested with the following block.

See here: https://github.com/garethjns/kaggle-football/blob/main/scripts/debug_agent.py for a version that can be used to run locally, but also maintain debugability.

In [ ]:
from kaggle_environments import make

env = make("football", configuration={"save_video": True,
                                      "scenario_name": "11_vs_11_kaggle"})

# Define players
left_player = "submission.py"  # A custom agent, eg. random_agent.py or example_agent.py
right_player = "run_right"  # eg. A built in 'AI' agent or the agent again


output = env.run([left_player, right_player])

print(f"Final score: {sum([r['reward'] for r in output[0]])} : {sum([r['reward'] for r in output[1]])}")

env.render(mode="human", width=800, height=600)

# Conclusions and next steps

The DQN performs currently extremely poorly, which is expected given the totally inadequate training done so far here. However, it seems to be running, so that's a start.

A few things to try:
  - Increase number of training iterations
  - Increase size of replay buffer
  - Increase number of units in network (set unit_scale higher)
  - Increase depth of NN (modify architecture)

# Appendix

## Training with reinforcement-learning-keras

Follows the same logic as above, but is a bit more fancy. https://github.com/garethjns/reinforcement-learning-keras

In [ ]:
!pip install reinforcement_learning_keras==0.6.0

In [ ]:
import gfootball

from reinforcement_learning_keras.agents.components.history.training_history import TrainingHistory
from reinforcement_learning_keras.agents.components.replay_buffers.continuous_buffer import ContinuousBuffer
from reinforcement_learning_keras.agents.models.dense_nn import DenseNN
from reinforcement_learning_keras.agents.q_learning.deep_q_agent import DeepQAgent
from reinforcement_learning_keras.agents.q_learning.exploration.epsilon_greedy import EpsilonGreedy


agent = DeepQAgent(name='deep_q',
                   model_architecture=DenseNN(observation_shape=(115,), n_actions=19),
                   replay_buffer=ContinuousBuffer(buffer_size=30000),
                   env_spec="GFootball-11_vs_11_kaggle-simple115v2-v0",
                   eps=EpsilonGreedy(eps_initial=0.9, decay=0.0001, eps_min=0.01, 
                                     decay_schedule='linear'),
                   training_history=TrainingHistory(agent_name='deep_q', rolling_average=1,
                                                    plotting_on=True, plot_every=5))

agent.train(verbose=True, render=False,
            n_episodes=10, max_episode_steps=3000, update_every=2,
            checkpoint_every=1)